In [1385]:
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
import os
import glob
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib.dates as md
import warnings
from scipy.interpolate import interp2d
warnings.filterwarnings('ignore')
#import datetime
import scipy.ndimage as ndimage
from matplotlib import cm
import geopy.distance
#import matplotlib as mpl

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import leafmap
import plotly.express as px
from mpl_toolkits.basemap import Basemap

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import pysplit as pys
import matplotlib.transforms as mtransforms

In [1386]:
def simple_read(path):
    with open(path) as f:
        first_line = f.readline()
        header_line = int(first_line[0:-2].split(",")[0])-1
        #all_lines = f.readlines()
    data = pd.read_csv(path, sep=',', skiprows=header_line)
    data.columns = [i.strip() for i in data.columns]
    #date = all_lines[5][0:10].split(',')
    #dt = datetime(int(date[0]), int(date[1]), int(date[2]))

    acc = 0
    boo = False
    for letter in path:
        if letter == '2':
            boo = True
        elif boo and letter == '0':
            acc -= 1
            break
        acc += 1
    #print(path[:acc+4], path[acc+4:acc+6], path[acc+6:acc+8])
    dt = datetime(int(path[acc:acc+4]), int(path[acc+4:acc+6]), int(path[acc+6:acc+8]))
    
    for column in data.keys():
        if 'Time' in column:
            data[column] = dt + pd.to_timedelta(data[column], unit='seconds')
    return data.replace(-999999, np.NaN)

In [1387]:
def add_sum(data, s):
    data = pd.merge_asof(data, s, left_index=True, right_index=True)
    return data

In [1388]:
LAS22 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220602_RA_L1_20230131T122324.ict')
start_index = np.where(LAS22["Time_Start"] == pd.to_datetime("2022-06-02 12:58:55"))
stop_index = np.where(LAS22["Time_Start"] == pd.to_datetime("2022-06-02 13:10:00"))


LAS22_subset = LAS22[start_index[0][0]:stop_index[0][0]]
LAS22_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
5981,2022-06-02 12:58:55,2022-06-02 12:58:56,115.705,293.02,55.75,6.81,240.32,331.92,375.49,508.39,...,0.00,17.30,17.32,25.96,0.0,0.00,10.39,0.00,0.0,1.103
5982,2022-06-02 12:58:56,2022-06-02 12:58:57,115.605,298.92,81.83,21.50,288.96,399.60,522.31,448.96,...,0.00,0.00,17.35,13.00,0.0,10.40,0.00,0.00,10.4,1.103
5983,2022-06-02 12:58:57,2022-06-02 12:58:58,114.775,281.59,78.28,14.64,239.79,430.79,312.07,536.89,...,25.87,0.00,17.27,0.00,0.0,20.71,20.72,0.00,0.0,1.104
5984,2022-06-02 12:58:58,2022-06-02 12:58:59,113.025,236.74,53.54,10.40,431.36,298.06,332.64,387.90,...,0.00,34.51,0.00,12.94,0.0,0.00,0.00,10.35,0.0,1.103
5985,2022-06-02 12:58:59,2022-06-02 12:59:00,110.485,232.14,30.10,1.36,262.30,164.74,434.09,267.06,...,0.00,0.00,17.19,0.00,0.0,0.00,0.00,0.00,0.0,1.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6641,2022-06-02 13:09:55,2022-06-02 13:09:56,3810.860,213.92,14.12,0.42,986.18,1129.56,626.76,538.28,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,1.589
6642,2022-06-02 13:09:56,2022-06-02 13:09:57,3811.410,163.86,10.53,0.32,792.83,730.67,625.60,447.97,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,1.589
6643,2022-06-02 13:09:57,2022-06-02 13:09:58,3812.340,206.88,13.70,0.42,1273.67,963.25,479.53,507.70,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,1.589
6644,2022-06-02 13:09:58,2022-06-02 13:09:59,3813.680,183.10,12.73,0.43,1130.71,698.68,543.42,209.43,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,1.589


In [1389]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS22_subset.drop(columns=columns_to_remove, inplace=True)

In [1390]:
SMPS22 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220602_RA_L1_20230131T122818.ict')
start_index = np.where(SMPS22["Time_Start"]==pd.to_datetime("2022-06-02 13:48:00"))
stop_index = np.where(SMPS22["Time_Start"]==pd.to_datetime("2022-06-02 14:10:00"))
    
SMPS22_subset = SMPS22[start_index[0][0]:stop_index[0][0]]
SMPS22_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
8926,2022-06-02 13:48:00,2022-06-02 13:48:01,1896.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8927,2022-06-02 13:48:01,2022-06-02 13:48:02,1897.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8928,2022-06-02 13:48:02,2022-06-02 13:48:03,1898.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8929,2022-06-02 13:48:03,2022-06-02 13:48:04,1898.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930,2022-06-02 13:48:04,2022-06-02 13:48:05,1899.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,2022-06-02 14:09:55,2022-06-02 14:09:56,2855.32,260.88,2.47,0.03,NaN,NaN,0.0,0.0,...,492.73,104.88,724.82,578.55,373.19,500.37,416.87,390.16,445.65,1.439
10242,2022-06-02 14:09:56,2022-06-02 14:09:57,2861.28,260.88,2.47,0.03,NaN,NaN,0.0,0.0,...,492.73,104.88,724.82,578.55,373.19,500.37,416.87,390.16,445.65,1.439
10243,2022-06-02 14:09:57,2022-06-02 14:09:58,2867.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10244,2022-06-02 14:09:58,2022-06-02 14:09:59,2872.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1391]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS22_subset.drop(columns=columns_to_remove, inplace=True)

In [1392]:
data = add_sum(LAS22_subset, SMPS22_subset)

In [1393]:
list(data)

['Time_Start',
 'Time_Stop',
 'GPS_Altitude_THORNHILL',
 'LAS_Bin01_ZIEMBA',
 'LAS_Bin02_ZIEMBA',
 'LAS_Bin03_ZIEMBA',
 'LAS_Bin04_ZIEMBA',
 'LAS_Bin05_ZIEMBA',
 'LAS_Bin06_ZIEMBA',
 'LAS_Bin07_ZIEMBA',
 'LAS_Bin08_ZIEMBA',
 'LAS_Bin09_ZIEMBA',
 'LAS_Bin10_ZIEMBA',
 'LAS_Bin11_ZIEMBA',
 'LAS_Bin12_ZIEMBA',
 'LAS_Bin13_ZIEMBA',
 'LAS_Bin14_ZIEMBA',
 'LAS_Bin15_ZIEMBA',
 'LAS_Bin16_ZIEMBA',
 'LAS_Bin17_ZIEMBA',
 'LAS_Bin18_ZIEMBA',
 'LAS_Bin19_ZIEMBA',
 'LAS_Bin20_ZIEMBA',
 'LAS_Bin21_ZIEMBA',
 'LAS_Bin22_ZIEMBA',
 'LAS_Bin23_ZIEMBA',
 'LAS_Bin24_ZIEMBA',
 'LAS_Bin25_ZIEMBA',
 'LAS_Bin26_ZIEMBA',
 'SMPS_Bin01_ZIEMBA',
 'SMPS_Bin02_ZIEMBA',
 'SMPS_Bin03_ZIEMBA',
 'SMPS_Bin04_ZIEMBA',
 'SMPS_Bin05_ZIEMBA',
 'SMPS_Bin06_ZIEMBA',
 'SMPS_Bin07_ZIEMBA',
 'SMPS_Bin08_ZIEMBA',
 'SMPS_Bin09_ZIEMBA',
 'SMPS_Bin10_ZIEMBA',
 'SMPS_Bin11_ZIEMBA',
 'SMPS_Bin12_ZIEMBA',
 'SMPS_Bin13_ZIEMBA',
 'SMPS_Bin14_ZIEMBA',
 'SMPS_Bin15_ZIEMBA',
 'SMPS_Bin16_ZIEMBA',
 'SMPS_Bin17_ZIEMBA',
 'SMPS_Bin18_ZIEMBA',
 '

In [1394]:
data

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
5981,2022-06-02 12:58:55,2022-06-02 12:58:56,115.705,240.32,331.92,375.49,508.39,688.44,907.78,950.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5982,2022-06-02 12:58:56,2022-06-02 12:58:57,115.605,288.96,399.60,522.31,448.96,574.00,705.26,1197.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5983,2022-06-02 12:58:57,2022-06-02 12:58:58,114.775,239.79,430.79,312.07,536.89,762.93,831.49,731.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5984,2022-06-02 12:58:58,2022-06-02 12:58:59,113.025,431.36,298.06,332.64,387.90,552.27,572.57,487.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5985,2022-06-02 12:58:59,2022-06-02 12:59:00,110.485,262.30,164.74,434.09,267.06,644.74,533.01,889.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6641,2022-06-02 13:09:55,2022-06-02 13:09:56,3810.860,986.18,1129.56,626.76,538.28,287.06,166.77,190.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6642,2022-06-02 13:09:56,2022-06-02 13:09:57,3811.410,792.83,730.67,625.60,447.97,210.04,185.15,81.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6643,2022-06-02 13:09:57,2022-06-02 13:09:58,3812.340,1273.67,963.25,479.53,507.70,248.35,259.15,81.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6644,2022-06-02 13:09:58,2022-06-02 13:09:59,3813.680,1130.71,698.68,543.42,209.43,440.02,185.35,163.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1395]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(data.GPS_Altitude_THORNHILL.min(), data.GPS_Altitude_THORNHILL.max(), nbins+1)
data['Bins'] = pd.cut(data['GPS_Altitude_THORNHILL'], bins_1000)
data_1000 = data.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_1000a = data_1000[start_index:stop_index]
DAY1000_subset_1000a


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(105.985, 143.17]",2022-06-02 12:59:00.500,2022-06-02 12:59:01.500,113.9000,251.310,266.525,322.825,357.485,534.465,628.300,732.270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(143.17, 180.354]",2022-06-02 12:59:09.500,2022-06-02 12:59:10.500,160.4275,227.945,215.805,291.435,268.795,429.540,573.585,585.340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(180.354, 217.539]",2022-06-02 12:59:14.500,2022-06-02 12:59:15.500,199.8250,167.700,299.070,292.085,329.020,448.975,630.025,705.365,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(217.539, 254.723]",2022-06-02 12:59:20.500,2022-06-02 12:59:21.500,234.4575,204.510,248.745,345.775,285.185,535.540,685.960,799.030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(254.723, 291.908]",2022-06-02 12:59:26.000,2022-06-02 12:59:27.000,276.6850,216.140,232.550,334.480,416.160,438.380,792.010,705.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(291.908, 329.092]",2022-06-02 12:59:32.000,2022-06-02 12:59:33.000,304.7050,168.360,265.780,290.330,269.340,459.060,592.570,621.470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(329.092, 366.277]",2022-06-02 12:59:37.000,2022-06-02 12:59:38.000,345.2250,216.640,199.710,271.880,358.980,497.670,593.400,870.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(366.277, 403.461]",2022-06-02 12:59:40.500,2022-06-02 12:59:41.500,383.8625,215.320,280.905,260.645,328.430,563.660,703.970,664.435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(403.461, 440.646]",2022-06-02 12:59:45.500,2022-06-02 12:59:46.500,420.9575,179.675,232.285,302.175,328.590,468.330,657.300,718.195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1396]:
DAY1000_subset_1000a = DAY1000_subset_1000a.reset_index(drop=True)

In [1397]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_3000a = data_1000[start_index:stop_index]
DAY3000_subset_3000a

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(998.414, 1035.599]",2022-06-02 13:00:58.500,2022-06-02 13:00:59.500,1020.465,203.815,198.785,322.490,238.615,362.955,498.885,474.940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1035.599, 1072.783]",2022-06-02 13:01:03.000,2022-06-02 13:01:04.000,1057.400,216.480,232.960,269.150,328.970,458.460,516.820,485.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1072.783, 1109.968]",2022-06-02 13:01:08.000,2022-06-02 13:01:09.000,1089.620,215.860,231.820,229.970,418.960,362.300,426.930,353.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1109.968, 1147.152]",2022-06-02 13:01:13.000,2022-06-02 13:01:14.000,1128.500,215.800,232.520,250.460,238.680,285.960,332.590,379.380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1147.152, 1184.337]",2022-06-02 13:01:17.500,2022-06-02 13:01:18.500,1167.055,180.270,249.175,250.975,254.255,296.760,371.115,366.875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3266.672, 3303.856]",2022-06-02 13:06:03.500,2022-06-02 13:06:04.500,3284.205,661.360,449.255,396.495,284.805,287.310,176.080,149.595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3303.856, 3341.041]",2022-06-02 13:06:09.000,2022-06-02 13:06:10.000,3322.510,721.210,565.350,438.450,329.110,267.800,148.310,108.650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3341.041, 3378.225]",2022-06-02 13:06:13.500,2022-06-02 13:06:14.500,3359.940,913.680,614.850,594.945,553.380,315.570,203.740,190.170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1398]:
DAY3000_subset_3000a = DAY3000_subset_3000a.reset_index(drop=True)

In [1399]:
DAY3000_subset_3000a

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-02 13:00:58.500,2022-06-02 13:00:59.500,1020.465,203.815,198.785,322.490,238.615,362.955,498.885,474.940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-02 13:01:03.000,2022-06-02 13:01:04.000,1057.400,216.480,232.960,269.150,328.970,458.460,516.820,485.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-02 13:01:08.000,2022-06-02 13:01:09.000,1089.620,215.860,231.820,229.970,418.960,362.300,426.930,353.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-02 13:01:13.000,2022-06-02 13:01:14.000,1128.500,215.800,232.520,250.460,238.680,285.960,332.590,379.380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-02 13:01:17.500,2022-06-02 13:01:18.500,1167.055,180.270,249.175,250.975,254.255,296.760,371.115,366.875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2022-06-02 13:06:03.500,2022-06-02 13:06:04.500,3284.205,661.360,449.255,396.495,284.805,287.310,176.080,149.595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,2022-06-02 13:06:09.000,2022-06-02 13:06:10.000,3322.510,721.210,565.350,438.450,329.110,267.800,148.310,108.650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,2022-06-02 13:06:13.500,2022-06-02 13:06:14.500,3359.940,913.680,614.850,594.945,553.380,315.570,203.740,190.170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,2022-06-02 13:06:18.000,2022-06-02 13:06:19.000,3397.350,1203.030,731.270,731.390,568.900,439.180,240.520,190.190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1400]:
'''nbins = 5
bins = np.linspace(DAY3000_subset_3000a.GPS_Altitude_THORNHILL.min(), DAY3000_subset_3000a.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY3000_subset_3000a['Bins'] = pd.cut(DAY3000_subset_3000a['GPS_Altitude_THORNHILL'], bins)
DAY3000_subset_3000a_median = DAY3000_subset_3000a.groupby('Bins').median()
DAY3000_subset_3000a_median
DAY3000_subset_3000a_25 = DAY3000_subset_3000a_median - DAY3000_subset_3000a.groupby('Bins').quantile(.25)
DAY3000_subset_3000a_25
DAY3000_subset_3000a_75 = DAY3000_subset_3000a.groupby('Bins').quantile(.75) - DAY3000_subset_3000a_median
DAY3000_subset_3000a_75'''

"nbins = 5\nbins = np.linspace(DAY3000_subset_3000a.GPS_Altitude_THORNHILL.min(), DAY3000_subset_3000a.GPS_Altitude_THORNHILL.max(), nbins+1)\nDAY3000_subset_3000a['Bins'] = pd.cut(DAY3000_subset_3000a['GPS_Altitude_THORNHILL'], bins)\nDAY3000_subset_3000a_median = DAY3000_subset_3000a.groupby('Bins').median()\nDAY3000_subset_3000a_median\nDAY3000_subset_3000a_25 = DAY3000_subset_3000a_median - DAY3000_subset_3000a.groupby('Bins').quantile(.25)\nDAY3000_subset_3000a_25\nDAY3000_subset_3000a_75 = DAY3000_subset_3000a.groupby('Bins').quantile(.75) - DAY3000_subset_3000a_median\nDAY3000_subset_3000a_75"

In [1401]:
'''DAY3000_subset_3000a_median
    
value22a = []
uevalue = []
levalue = []

for key in DAY3000_subset_3000a_median.keys():
    if 'Bin' in key:
        value22a.append(DAY3000_subset_3000a_median[key].T)
        uevalue.append(DAY3000_subset_3000a_75[key].T)
        levalue.append(DAY3000_subset_3000a_25[key].T)
value22a = np.array(value22a)
uevalue = np.array(uevalue)
levalue = np.array(levalue)
value22a[value22a==0] = 1
uevalue[uevalue==0] = 1
levalue[levalue==0] = 1
    
LAS_mid = [100.0, 112.2, 125.9, 141.3, 158.5, 177.8, 199.5, 223.9, 251.2, 281.8, 316.2, 354.8, 398.1, 446.7, 501.2, 562.3, 631.0, 707.9, 794.3, 891.3, 1000.0, 1258.9, 1584.9, 1995.3, 2511.9, 3162.3]
SMPS_mid = [3.16, 3.55, 3.98, 4.47, 5.01, 5.62, 6.31, 7.08, 7.94, 8.91, 10.0, 11.2, 12.6, 14.1, 15.8, 17.8, 20.0, 22.4, 25.1, 28.2, 31.6, 35.5, 39.8, 44.7, 50.1, 56.2, 63.1, 70.8, 79.4, 89.1]
all_mid = LAS_mid + SMPS_mid'''

"DAY3000_subset_3000a_median\n    \nvalue22a = []\nuevalue = []\nlevalue = []\n\nfor key in DAY3000_subset_3000a_median.keys():\n    if 'Bin' in key:\n        value22a.append(DAY3000_subset_3000a_median[key].T)\n        uevalue.append(DAY3000_subset_3000a_75[key].T)\n        levalue.append(DAY3000_subset_3000a_25[key].T)\nvalue22a = np.array(value22a)\nuevalue = np.array(uevalue)\nlevalue = np.array(levalue)\nvalue22a[value22a==0] = 1\nuevalue[uevalue==0] = 1\nlevalue[levalue==0] = 1\n    \nLAS_mid = [100.0, 112.2, 125.9, 141.3, 158.5, 177.8, 199.5, 223.9, 251.2, 281.8, 316.2, 354.8, 398.1, 446.7, 501.2, 562.3, 631.0, 707.9, 794.3, 891.3, 1000.0, 1258.9, 1584.9, 1995.3, 2511.9, 3162.3]\nSMPS_mid = [3.16, 3.55, 3.98, 4.47, 5.01, 5.62, 6.31, 7.08, 7.94, 8.91, 10.0, 11.2, 12.6, 14.1, 15.8, 17.8, 20.0, 22.4, 25.1, 28.2, 31.6, 35.5, 39.8, 44.7, 50.1, 56.2, 63.1, 70.8, 79.4, 89.1]\nall_mid = LAS_mid + SMPS_mid"

In [1402]:
'''data_75

uevalue = []
for key in data_75.keys():
    if 'Bin' in key:
        uevalue.append(data_75[key].T)
uevalue = np.array(uevalue)
uevalue[uevalue==0] = 1


'''


"data_75\n\nuevalue = []\nfor key in data_75.keys():\n    if 'Bin' in key:\n        uevalue.append(data_75[key].T)\nuevalue = np.array(uevalue)\nuevalue[uevalue==0] = 1\n\n\n"

In [1403]:
'''data_25

levalue = []
for key in data_25.keys():
    if 'Bin' in key:
        levalue.append(data_25[key].T)
levalue = np.array(levalue)
levalue[levalue==0] = 1'''





"data_25\n\nlevalue = []\nfor key in data_25.keys():\n    if 'Bin' in key:\n        levalue.append(data_25[key].T)\nlevalue = np.array(levalue)\nlevalue[levalue==0] = 1"

In [1404]:
'''data_median
    
value22a = []

for key in data_median.keys():
    if 'Bin' in key:
        value22a.append(data_median[key].T)
value22a = np.array(value22a)
value22a[value22a==0] = 1'''

"data_median\n    \nvalue22a = []\n\nfor key in data_median.keys():\n    if 'Bin' in key:\n        value22a.append(data_median[key].T)\nvalue22a = np.array(value22a)\nvalue22a[value22a==0] = 1"

In [1405]:
'''data_median
    
value22a = []
uevalue = []
levalue = []

for key in data_median.keys():
    if 'Bin' in key:
        value22a.append(data_median[key].T)
        uevalue.append(data_75[key].T)
        levalue.append(data_25[key].T)
value22a = np.array(value22a)
uevalue = np.array(uevalue)
levalue = np.array(levalue)
value22a[value22a==0] = 1
#uevalue[uevalue==0] = 1
#levalue[levalue==0] = 1'''
    
LAS_mid = [100.0, 112.2, 125.9, 141.3, 158.5, 177.8, 199.5, 223.9, 251.2, 281.8, 316.2, 354.8, 398.1, 446.7, 501.2, 562.3, 631.0, 707.9, 794.3, 891.3, 1000.0, 1258.9, 1584.9, 1995.3, 2511.9, 3162.3]
SMPS_mid = [3.16, 3.55, 3.98, 4.47, 5.01, 5.62, 6.31, 7.08, 7.94, 8.91, 10.0, 11.2, 12.6, 14.1, 15.8, 17.8, 20.0, 22.4, 25.1, 28.2, 31.6, 35.5, 39.8, 44.7, 50.1, 56.2, 63.1, 70.8, 79.4, 89.1]
all_mid = LAS_mid + SMPS_mid

In [1406]:
all_mid

[100.0,
 112.2,
 125.9,
 141.3,
 158.5,
 177.8,
 199.5,
 223.9,
 251.2,
 281.8,
 316.2,
 354.8,
 398.1,
 446.7,
 501.2,
 562.3,
 631.0,
 707.9,
 794.3,
 891.3,
 1000.0,
 1258.9,
 1584.9,
 1995.3,
 2511.9,
 3162.3,
 3.16,
 3.55,
 3.98,
 4.47,
 5.01,
 5.62,
 6.31,
 7.08,
 7.94,
 8.91,
 10.0,
 11.2,
 12.6,
 14.1,
 15.8,
 17.8,
 20.0,
 22.4,
 25.1,
 28.2,
 31.6,
 35.5,
 39.8,
 44.7,
 50.1,
 56.2,
 63.1,
 70.8,
 79.4,
 89.1]

In [1407]:
len(all_mid)

56

In [1408]:
'''fig, ax = plt.subplots()
plt.rcParams['font.size'] = '10'
ax.errorbar(all_mid, value22a, xerr=[levalue, uevalue], marker='o', linestyle='-', capsize=4, alpha=.6, color="darkorange")
ax.set_ylabel('Altitude (m)', labelpad=11)
ax.set_xscale('log')
ax.legend(bbox_to_anchor=(1.15, 1), loc = 'right', fontsize='medium')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)
fig.text(0.02, 0.98, '06/02_12:58:13:10 UTC', ha='left', va='top', fontsize=10)
# Set the title
fig.suptitle('number concentration', fontsize=16)

plt.show()'''


'fig, ax = plt.subplots()\nplt.rcParams[\'font.size\'] = \'10\'\nax.errorbar(all_mid, value22a, xerr=[levalue, uevalue], marker=\'o\', linestyle=\'-\', capsize=4, alpha=.6, color="darkorange")\nax.set_ylabel(\'Altitude (m)\', labelpad=11)\nax.set_xscale(\'log\')\nax.legend(bbox_to_anchor=(1.15, 1), loc = \'right\', fontsize=\'medium\')\n\n# Adjust the spacing between subplots\nplt.subplots_adjust(wspace=0.3)\nfig.text(0.02, 0.98, \'06/02_12:58:13:10 UTC\', ha=\'left\', va=\'top\', fontsize=10)\n# Set the title\nfig.suptitle(\'number concentration\', fontsize=16)\n\nplt.show()'

In [1409]:
LAS = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220602_RA_L1_20230131T122324.ict')
start_index = np.where(LAS["Time_Start"]==pd.to_datetime("2022-06-02 13:48:00"))
stop_index = np.where(LAS["Time_Start"]==pd.to_datetime("2022-06-02 14:10:00"))
    
LAS_subset = LAS[start_index[0][0]:stop_index[0][0]]
LAS_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
8926,2022-06-02 13:48:00,2022-06-02 13:48:01,1896.68,125.18,16.58,1.07,360.53,232.87,208.88,299.46,...,0.00,0.00,34.65,0.00,0.0,0.0,0.0,0.0,0.0,1.301
8927,2022-06-02 13:48:01,2022-06-02 13:48:02,1897.28,105.98,13.16,0.63,144.19,332.14,208.62,89.50,...,25.94,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.301
8928,2022-06-02 13:48:02,2022-06-02 13:48:03,1898.01,108.47,11.92,0.69,408.32,232.86,229.87,149.47,...,0.00,0.00,0.00,12.97,0.0,0.0,0.0,0.0,0.0,1.301
8929,2022-06-02 13:48:03,2022-06-02 13:48:04,1898.83,127.25,14.64,0.71,504.51,232.03,208.89,239.23,...,0.00,17.29,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.302
8930,2022-06-02 13:48:04,2022-06-02 13:48:05,1899.75,133.17,12.27,0.41,384.68,332.31,292.10,268.94,...,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,2022-06-02 14:09:55,2022-06-02 14:09:56,2855.32,152.94,14.05,0.52,696.81,398.76,292.34,448.58,...,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.445
10242,2022-06-02 14:09:56,2022-06-02 14:09:57,2861.28,150.62,11.90,0.41,865.52,431.41,396.28,328.90,...,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.446
10243,2022-06-02 14:09:57,2022-06-02 14:09:58,2867.15,164.86,15.26,0.60,697.16,531.94,520.54,299.09,...,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.447
10244,2022-06-02 14:09:58,2022-06-02 14:09:59,2872.99,194.08,15.05,0.50,818.04,731.78,585.09,328.91,...,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.448


In [1410]:
SMPS = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220602_RA_L1_20230131T122818.ict')
start_index = np.where(SMPS["Time_Start"]==pd.to_datetime("2022-06-02 13:48:00"))
stop_index = np.where(SMPS["Time_Start"]==pd.to_datetime("2022-06-02 14:10:00"))
    
SMPS_subset = SMPS[start_index[0][0]:stop_index[0][0]]
SMPS_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
8926,2022-06-02 13:48:00,2022-06-02 13:48:01,1896.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8927,2022-06-02 13:48:01,2022-06-02 13:48:02,1897.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8928,2022-06-02 13:48:02,2022-06-02 13:48:03,1898.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8929,2022-06-02 13:48:03,2022-06-02 13:48:04,1898.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930,2022-06-02 13:48:04,2022-06-02 13:48:05,1899.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,2022-06-02 14:09:55,2022-06-02 14:09:56,2855.32,260.88,2.47,0.03,NaN,NaN,0.0,0.0,...,492.73,104.88,724.82,578.55,373.19,500.37,416.87,390.16,445.65,1.439
10242,2022-06-02 14:09:56,2022-06-02 14:09:57,2861.28,260.88,2.47,0.03,NaN,NaN,0.0,0.0,...,492.73,104.88,724.82,578.55,373.19,500.37,416.87,390.16,445.65,1.439
10243,2022-06-02 14:09:57,2022-06-02 14:09:58,2867.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10244,2022-06-02 14:09:58,2022-06-02 14:09:59,2872.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1411]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS_subset.drop(columns=columns_to_remove, inplace=True)

In [1412]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS_subset.drop(columns=columns_to_remove, inplace=True)

In [1413]:
data2 = add_sum(LAS_subset, SMPS_subset)

In [1414]:
#binning data2 into groups
nbins = 100
bins_1000 = np.linspace(data2.GPS_Altitude_THORNHILL.min(), data2.GPS_Altitude_THORNHILL.max(), nbins+1)
data2['Bins'] = pd.cut(data2['GPS_Altitude_THORNHILL'], bins_1000)
data2_1000 = data2.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data2_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data2_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_1000b = data2_1000[start_index:stop_index]
DAY1000_subset_1000b


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(106.28, 134.006]",2022-06-02 13:58:08.000,2022-06-02 13:58:09.000,122.4800,240.250,264.340,334.080,386.740,515.690,664.090,705.970,...,301.950,347.570,211.570,155.410,416.910,216.160,512.730,257.670,217.650,292.380
"(134.006, 161.732]",2022-06-02 13:58:49.000,2022-06-02 13:58:50.000,149.2400,215.420,298.920,313.640,447.540,497.860,628.590,759.810,...,289.860,43.850,294.050,207.090,513.670,410.850,306.170,219.040,217.680,160.000
"(161.732, 189.459]",2022-06-02 13:59:02.000,2022-06-02 13:59:03.000,174.5650,216.140,265.030,313.360,359.000,478.620,630.780,733.300,...,289.860,43.850,294.050,207.090,513.670,410.850,306.170,219.040,217.680,160.000
"(189.459, 217.185]",2022-06-02 13:57:08.000,2022-06-02 13:57:09.000,202.3350,215.360,332.400,393.910,299.230,460.240,705.940,543.100,...,189.860,259.700,470.830,439.770,310.460,25.560,75.810,175.580,79.360,346.860
"(217.185, 244.911]",2022-06-02 13:57:01.500,2022-06-02 13:57:02.500,231.2250,192.270,265.305,312.590,343.205,505.860,657.250,813.300,...,189.860,259.700,470.830,439.770,310.460,25.560,75.810,175.580,79.360,346.860
"(244.911, 272.637]",2022-06-02 13:58:08.000,2022-06-02 13:58:09.000,260.1525,228.125,265.155,344.760,328.770,486.745,610.700,691.710,...,342.225,304.270,526.680,219.885,245.270,89.865,130.130,213.865,202.485,344.440
"(272.637, 300.363]",2022-06-02 13:59:28.000,2022-06-02 13:59:29.000,282.5850,238.870,297.150,271.100,417.620,437.850,647.880,706.040,...,189.860,259.700,470.830,439.770,310.460,25.560,75.810,175.580,79.360,346.860
"(300.363, 328.09]",2022-06-02 13:56:41.500,2022-06-02 13:56:42.500,312.9625,252.800,216.010,313.385,374.225,467.620,667.355,676.875,...,494.590,348.840,582.530,0.000,180.080,154.170,184.450,252.150,325.610,342.020
"(328.09, 355.816]",2022-06-02 13:56:33.500,2022-06-02 13:56:34.500,339.6750,204.285,282.855,322.940,403.050,486.945,573.520,745.395,...,494.590,348.840,582.530,0.000,180.080,154.170,184.450,252.150,325.610,342.020


In [1415]:
DAY1000_subset_1000b = DAY1000_subset_1000b.reset_index(drop=True)

In [1416]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data2_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data2_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_3000b = data2_1000[start_index:stop_index]
DAY3000_subset_3000b

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(993.518, 1021.245]",2022-06-02 14:02:19.500,2022-06-02 14:02:20.500,1005.710,276.810,265.660,335.030,373.770,478.755,529.455,597.820,...,322.73,287.99,462.57,269.28,273.73,125.89,296.45,287.68,172.02,391.84
"(1021.245, 1048.971]",2022-06-02 14:02:24.500,2022-06-02 14:02:25.500,1036.645,263.660,349.295,312.755,373.590,419.405,600.435,582.760,...,322.73,287.99,462.57,269.28,273.73,125.89,296.45,287.68,172.02,391.84
"(1048.971, 1076.697]",2022-06-02 14:02:31.500,2022-06-02 14:02:32.500,1060.535,263.960,315.845,354.480,388.695,487.510,536.855,582.795,...,322.73,287.99,462.57,269.28,273.73,125.89,296.45,287.68,172.02,391.84
"(1076.697, 1104.423]",2022-06-02 14:02:37.500,2022-06-02 14:02:38.500,1089.530,275.645,314.090,302.105,372.595,438.710,535.025,475.350,...,322.73,287.99,462.57,269.28,273.73,125.89,296.45,287.68,172.02,391.84
"(1104.423, 1132.149]",2022-06-02 14:02:43.000,2022-06-02 14:02:44.000,1119.480,240.420,266.050,313.700,358.750,420.340,444.970,407.530,...,322.73,287.99,462.57,269.28,273.73,125.89,296.45,287.68,172.02,391.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2712.543, 2740.269]",2022-06-02 14:09:22.000,2022-06-02 14:09:23.000,2729.930,793.910,632.340,605.470,478.350,344.090,333.390,325.320,...,401.49,492.73,104.88,724.82,578.55,373.19,500.37,416.87,390.16,445.65
"(2740.269, 2767.995]",2022-06-02 14:09:30.500,2022-06-02 14:09:31.500,2751.515,565.200,498.190,500.740,298.840,411.070,305.355,149.275,...,401.49,492.73,104.88,724.82,578.55,373.19,500.37,416.87,390.16,445.65
"(2767.995, 2795.721]",2022-06-02 14:09:38.000,2022-06-02 14:09:39.000,2784.750,624.470,499.640,459.790,358.570,363.310,278.420,217.120,...,401.49,492.73,104.88,724.82,578.55,373.19,500.37,416.87,390.16,445.65


In [1417]:
LAS2b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220602_RA_L2_20230131T122324.ict')
start_index = np.where(LAS2b["Time_Start"]==pd.to_datetime("2022-06-02 18:36:00"))
stop_index = np.where(LAS2b["Time_Start"]==pd.to_datetime("2022-06-02 18:53:00"))
    
LAS2b_subset = LAS2b[start_index[0][0]:stop_index[0][0]]
LAS2b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
9180,2022-06-02 18:36:00,2022-06-02 18:36:01,191.225,220.35,28.35,1.19,119.93,166.08,375.02,358.28,...,0.0,17.27,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.110
9181,2022-06-02 18:36:01,2022-06-02 18:36:02,193.480,248.28,35.68,1.86,215.95,232.26,250.41,477.42,...,0.0,0.00,17.29,12.96,0.00,0.00,0.0,0.0,0.00,1.110
9182,2022-06-02 18:36:02,2022-06-02 18:36:03,195.600,245.18,65.75,18.65,337.39,166.13,460.06,419.15,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,10.41,1.110
9183,2022-06-02 18:36:03,2022-06-02 18:36:04,197.010,269.44,51.18,5.11,167.38,264.14,415.64,327.20,...,25.8,0.00,0.00,12.92,10.33,10.33,0.0,0.0,0.00,1.111
9184,2022-06-02 18:36:04,2022-06-02 18:36:05,197.440,255.98,32.04,1.20,96.52,400.72,356.44,390.39,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,2022-06-02 18:52:55,2022-06-02 18:52:56,3678.610,290.88,14.77,0.38,2140.67,1429.70,941.74,658.44,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.571
10196,2022-06-02 18:52:56,2022-06-02 18:52:57,3684.900,287.82,16.20,0.46,2044.58,1329.33,981.41,508.76,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.572
10197,2022-06-02 18:52:57,2022-06-02 18:52:58,3691.340,299.42,16.67,0.46,1969.50,1560.97,834.51,567.92,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.574
10198,2022-06-02 18:52:58,2022-06-02 18:52:59,3697.790,306.24,16.67,0.44,1875.74,1694.55,982.46,627.50,...,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.576


In [1418]:
SMPS2b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220602_RA_L2_20230131T122818.ict')
start_index = np.where(SMPS2b["Time_Start"]==pd.to_datetime("2022-06-02 18:36:00"))
stop_index = np.where(SMPS2b["Time_Start"]==pd.to_datetime("2022-06-02 18:53:00"))
    
SMPS2b_subset = SMPS2b[start_index[0][0]:stop_index[0][0]]
SMPS2b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
9180,2022-06-02 18:36:00,2022-06-02 18:36:01,191.225,342.66,3.17,0.03,0.0,0.0,0.0,0.0,...,442.45,836.26,728.92,603.14,659.68,799.43,334.17,797.47,291.89,1.113
9181,2022-06-02 18:36:01,2022-06-02 18:36:02,193.480,342.66,3.17,0.03,0.0,0.0,0.0,0.0,...,442.45,836.26,728.92,603.14,659.68,799.43,334.17,797.47,291.89,1.113
9182,2022-06-02 18:36:02,2022-06-02 18:36:03,195.600,342.66,3.17,0.03,0.0,0.0,0.0,0.0,...,442.45,836.26,728.92,603.14,659.68,799.43,334.17,797.47,291.89,1.113
9183,2022-06-02 18:36:03,2022-06-02 18:36:04,197.010,342.66,3.17,0.03,0.0,0.0,0.0,0.0,...,442.45,836.26,728.92,603.14,659.68,799.43,334.17,797.47,291.89,1.113
9184,2022-06-02 18:36:04,2022-06-02 18:36:05,197.440,342.66,3.17,0.03,0.0,0.0,0.0,0.0,...,442.45,836.26,728.92,603.14,659.68,799.43,334.17,797.47,291.89,1.113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,2022-06-02 18:52:55,2022-06-02 18:52:56,3678.610,1590.40,7.05,0.07,NaN,NaN,0.0,0.0,...,930.60,1062.13,2239.70,1562.11,1466.99,1065.20,1175.79,941.18,723.94,1.567
10196,2022-06-02 18:52:56,2022-06-02 18:52:57,3684.900,1590.40,7.05,0.07,NaN,NaN,0.0,0.0,...,930.60,1062.13,2239.70,1562.11,1466.99,1065.20,1175.79,941.18,723.94,1.567
10197,2022-06-02 18:52:57,2022-06-02 18:52:58,3691.340,1590.40,7.05,0.07,NaN,NaN,0.0,0.0,...,930.60,1062.13,2239.70,1562.11,1466.99,1065.20,1175.79,941.18,723.94,1.567
10198,2022-06-02 18:52:58,2022-06-02 18:52:59,3697.790,1590.40,7.05,0.07,NaN,NaN,0.0,0.0,...,930.60,1062.13,2239.70,1562.11,1466.99,1065.20,1175.79,941.18,723.94,1.567


In [1419]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS2b_subset.drop(columns=columns_to_remove, inplace=True)

In [1420]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS2b_subset.drop(columns=columns_to_remove, inplace=True)

In [1421]:
data2b = add_sum(LAS2b_subset, SMPS2b_subset)

In [1422]:
#binning data2b into groups
nbins = 100
bins_1000 = np.linspace(data2b.GPS_Altitude_THORNHILL.min(), data2b.GPS_Altitude_THORNHILL.max(), nbins+1)
data2b['Bins'] = pd.cut(data2b['GPS_Altitude_THORNHILL'], bins_1000)
data2b_1000 = data2b.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data2b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data2b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_10002b = data2b_1000[start_index:stop_index]
DAY1000_subset_10002b


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(132.5, 168.217]",2022-06-02 18:36:33.500,2022-06-02 18:36:34.500,151.3675,216.650,265.895,314.440,328.465,496.290,613.220,719.120,...,362.25,792.91,775.32,813.69,798.69,566.21,441.46,422.50,275.32,159.79
"(168.217, 203.933]",2022-06-02 18:36:12.000,2022-06-02 18:36:13.000,186.7200,216.510,298.180,292.090,419.150,532.820,685.490,813.460,...,476.51,442.45,836.26,728.92,603.14,659.68,799.43,334.17,797.47,291.89
"(203.933, 239.65]",2022-06-02 18:37:02.000,2022-06-02 18:37:03.000,214.3750,169.160,265.700,313.860,240.150,421.450,609.650,762.410,...,362.25,792.91,775.32,813.69,798.69,566.21,441.46,422.50,275.32,159.79
"(239.65, 275.366]",2022-06-02 18:37:13.000,2022-06-02 18:37:14.000,260.1500,240.140,166.630,228.200,359.180,421.280,663.300,789.510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(275.366, 311.083]",2022-06-02 18:37:24.000,2022-06-02 18:37:25.000,289.4500,239.290,199.490,271.690,359.580,457.420,593.300,680.040,...,512.71,261.62,207.39,332.68,288.02,268.06,211.44,335.30,236.66,371.08
"(311.083, 346.8]",2022-06-02 18:37:38.000,2022-06-02 18:37:39.000,332.1600,215.390,266.060,312.400,358.300,400.860,609.980,621.800,...,512.71,261.62,207.39,332.68,288.02,268.06,211.44,335.30,236.66,371.08
"(346.8, 382.516]",2022-06-02 18:37:53.000,2022-06-02 18:37:54.000,362.9050,263.870,300.030,291.430,298.020,438.520,554.440,596.220,...,512.71,261.62,207.39,332.68,288.02,268.06,211.44,335.30,236.66,371.08
"(382.516, 418.233]",2022-06-02 18:38:07.000,2022-06-02 18:38:08.000,399.5450,191.930,231.690,312.310,358.460,457.690,498.810,516.940,...,512.71,261.62,207.39,332.68,288.02,268.06,211.44,335.30,236.66,371.08
"(418.233, 453.949]",2022-06-02 18:38:19.000,2022-06-02 18:38:20.000,432.9000,216.240,198.630,312.090,298.720,420.150,480.310,542.660,...,330.91,356.75,283.23,189.36,425.54,476.31,227.63,120.31,230.66,307.69


In [1423]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data2b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data2b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_30002b = data2b_1000[start_index:stop_index]
DAY3000_subset_30002b

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(989.698, 1025.415]",2022-06-02 18:41:23.000,2022-06-02 18:41:24.000,1010.300,312.540,232.280,291.190,329.340,400.290,535.710,515.250,...,334.16,368.32,521.46,399.78,401.48,253.63,361.03,219.08,430.2,297.60
"(1025.415, 1061.132]",2022-06-02 18:41:31.500,2022-06-02 18:41:32.500,1046.060,263.865,265.235,354.840,299.235,400.700,481.085,407.115,...,334.16,368.32,521.46,399.78,401.48,253.63,361.03,219.08,430.2,297.60
"(1061.132, 1096.848]",2022-06-02 18:41:40.500,2022-06-02 18:41:41.500,1080.155,300.125,298.655,303.120,343.975,343.700,416.675,503.170,...,334.16,368.32,521.46,399.78,401.48,253.63,361.03,219.08,430.2,297.60
"(1096.848, 1132.565]",2022-06-02 18:41:52.500,2022-06-02 18:41:53.500,1111.995,239.695,232.255,291.385,283.865,372.425,462.880,487.385,...,334.16,368.32,521.46,399.78,401.48,253.63,361.03,219.08,430.2,297.60
"(1132.565, 1168.281]",2022-06-02 18:42:04.500,2022-06-02 18:42:05.500,1150.090,275.925,347.850,271.685,344.195,344.020,462.470,393.650,...,334.16,368.32,521.46,399.78,401.48,253.63,361.03,219.08,430.2,297.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3275.561, 3311.277]",2022-06-02 18:51:48.500,2022-06-02 18:51:49.500,3293.650,1033.785,647.165,511.985,253.800,210.210,83.280,54.295,...,384.09,685.37,659.81,796.36,1685.83,5189.77,1100.07,513.69,732.7,648.43
"(3311.277, 3346.994]",2022-06-02 18:51:54.000,2022-06-02 18:51:55.000,3329.670,1273.420,697.470,585.300,239.350,191.420,92.630,81.560,...,384.09,685.37,659.81,796.36,1685.83,5189.77,1100.07,513.69,732.7,648.43
"(3346.994, 3382.711]",2022-06-02 18:51:59.000,2022-06-02 18:52:00.000,3366.390,1488.890,864.390,626.490,358.650,248.820,111.010,54.340,...,384.09,685.37,659.81,796.36,1685.83,5189.77,1100.07,513.69,732.7,648.43


In [1424]:
DAY1000_subset_10002b = DAY1000_subset_10002b.reset_index(drop=True)
DAY3000_subset_30002b = DAY3000_subset_30002b.reset_index(drop=True)

In [1425]:
LAS7 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220607_RA_L1_20230131T122325.ict')
start_index = np.where(LAS7["Time_Start"]==pd.to_datetime("2022-06-07 14:32:00"))
stop_index = np.where(LAS7["Time_Start"]==pd.to_datetime("2022-06-07 14:47:00"))
    
LAS7_subset = LAS7[start_index[0][0]:stop_index[0][0]]
LAS7_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10397,2022-06-07 14:32:00,2022-06-07 14:32:01,127.595,665.22,51.40,1.67,2279.85,2224.35,2480.28,1373.68,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,1.088
10398,2022-06-07 14:32:01,2022-06-07 14:32:02,126.585,705.23,58.10,2.47,2544.73,2488.52,2312.98,1941.12,...,0.00,0.00,34.57,0.00,0.00,0.00,0.00,0.0,0.0,1.088
10399,2022-06-07 14:32:02,2022-06-07 14:32:03,125.870,680.19,87.40,11.73,2046.82,2496.76,1902.85,2126.88,...,77.97,17.33,17.34,13.01,0.00,0.00,20.81,0.0,0.0,1.088
10400,2022-06-07 14:32:03,2022-06-07 14:32:04,125.955,604.75,68.55,8.24,2024.90,2132.99,1905.74,1768.90,...,0.00,0.00,17.37,0.00,0.00,10.41,10.42,0.0,0.0,1.088
10401,2022-06-07 14:32:04,2022-06-07 14:32:05,126.990,602.24,64.47,5.38,1725.31,2020.77,1746.92,1610.05,...,25.89,17.25,17.26,0.00,10.35,10.35,0.00,0.0,0.0,1.087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292,2022-06-07 14:46:55,2022-06-07 14:46:56,2800.660,198.96,10.79,0.27,1248.75,962.58,626.07,477.55,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,1.431
11293,2022-06-07 14:46:56,2022-06-07 14:46:57,2805.770,206.00,11.95,0.31,1154.89,965.32,689.43,538.90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,1.432
11294,2022-06-07 14:46:57,2022-06-07 14:46:58,2810.920,239.42,13.36,0.35,1923.92,764.03,670.17,537.95,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,1.433
11295,2022-06-07 14:46:58,2022-06-07 14:46:59,2816.140,196.67,10.42,0.26,1273.93,830.54,876.47,418.17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,1.434


In [1426]:
SMPS7 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220607_RA_L1_20230131T122819.ict')
start_index = np.where(SMPS7["Time_Start"]==pd.to_datetime("2022-06-07 14:32:00"))
stop_index = np.where(SMPS7["Time_Start"]==pd.to_datetime("2022-06-07 14:47:00"))
    
SMPS7_subset = SMPS7[start_index[0][0]:stop_index[0][0]]
SMPS7_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10397,2022-06-07 14:32:00,2022-06-07 14:32:01,127.595,835.74,9.46,0.11,0.0,0.0,0.0,0.0,...,1054.51,1371.55,1616.39,1793.3,1074.02,1993.01,1998.35,1970.72,1742.82,1.089
10398,2022-06-07 14:32:01,2022-06-07 14:32:02,126.585,835.74,9.46,0.11,0.0,0.0,0.0,0.0,...,1054.51,1371.55,1616.39,1793.3,1074.02,1993.01,1998.35,1970.72,1742.82,1.089
10399,2022-06-07 14:32:02,2022-06-07 14:32:03,125.870,835.74,9.46,0.11,0.0,0.0,0.0,0.0,...,1054.51,1371.55,1616.39,1793.3,1074.02,1993.01,1998.35,1970.72,1742.82,1.089
10400,2022-06-07 14:32:03,2022-06-07 14:32:04,125.955,835.74,9.46,0.11,0.0,0.0,0.0,0.0,...,1054.51,1371.55,1616.39,1793.3,1074.02,1993.01,1998.35,1970.72,1742.82,1.089
10401,2022-06-07 14:32:04,2022-06-07 14:32:05,126.990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292,2022-06-07 14:46:55,2022-06-07 14:46:56,2800.660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426
11293,2022-06-07 14:46:56,2022-06-07 14:46:57,2805.770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426
11294,2022-06-07 14:46:57,2022-06-07 14:46:58,2810.920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426
11295,2022-06-07 14:46:58,2022-06-07 14:46:59,2816.140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.426


In [1427]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS7_subset.drop(columns=columns_to_remove, inplace=True)
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS7_subset.drop(columns=columns_to_remove, inplace=True)

In [1428]:
data7 = add_sum(LAS7_subset, SMPS7_subset)

In [1429]:
#binning data7 into groups
nbins = 100
bins_1000 = np.linspace(data7.GPS_Altitude_THORNHILL.min(), data7.GPS_Altitude_THORNHILL.max(), nbins+1)
data7['Bins'] = pd.cut(data7['GPS_Altitude_THORNHILL'], bins_1000)
data7_1000 = data7.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data7_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data7_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_10007 = data7_1000[start_index:stop_index]
DAY1000_subset_10007


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(125.87, 152.826]",2022-06-07 14:32:05.500,2022-06-07 14:32:06.500,130.6300,2032.270,2075.820,1973.565,1659.225,1592.380,1063.325,694.175,...,536.42,1054.51,1371.55,1616.39,1793.30,1074.02,1993.01,1998.35,1970.72,1742.82
"(152.826, 179.781]",2022-06-07 14:32:14.000,2022-06-07 14:32:15.000,172.0250,2350.470,2165.020,2186.610,1768.040,1662.370,1203.370,758.770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(179.781, 206.737]",2022-06-07 14:32:27.500,2022-06-07 14:32:28.500,188.3900,2292.240,2206.645,2116.335,1849.310,1577.410,1138.130,760.205,...,917.52,868.04,1333.83,1222.31,1812.84,716.57,1338.19,2233.26,1809.81,1030.33
"(206.737, 233.692]",2022-06-07 14:32:40.000,2022-06-07 14:32:41.000,219.6400,2044.860,2218.550,1968.420,1525.650,1510.960,1003.890,892.980,...,917.52,868.04,1333.83,1222.31,1812.84,716.57,1338.19,2233.26,1809.81,1030.33
"(233.692, 260.648]",2022-06-07 14:32:48.000,2022-06-07 14:32:49.000,250.9800,2356.010,2315.050,2241.750,1945.240,1655.410,1059.260,677.630,...,917.52,868.04,1333.83,1222.31,1812.84,716.57,1338.19,2233.26,1809.81,1030.33
"(260.648, 287.604]",2022-06-07 14:33:01.000,2022-06-07 14:33:02.000,261.7850,2022.460,2259.770,2125.930,1791.030,1608.050,1152.210,790.120,...,917.52,868.04,1333.83,1222.31,1812.84,716.57,1338.19,2233.26,1809.81,1030.33
"(287.604, 314.559]",2022-06-07 14:33:13.500,2022-06-07 14:33:14.500,305.6750,2260.970,2181.115,2166.585,1900.700,1587.480,1295.980,867.705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(314.559, 341.515]",2022-06-07 14:33:26.000,2022-06-07 14:33:27.000,326.2850,2164.470,2227.970,2238.150,1996.600,1647.640,1202.120,866.450,...,363.33,802.84,1690.54,1829.29,1062.87,1316.77,1664.06,2025.02,2438.57,2253.22
"(341.515, 368.47]",2022-06-07 14:33:38.000,2022-06-07 14:33:39.000,353.5650,2117.180,2096.500,1955.260,1850.720,1623.230,1147.060,706.890,...,363.33,802.84,1690.54,1829.29,1062.87,1316.77,1664.06,2025.02,2438.57,2253.22


In [1430]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data7_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data7_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_30007 = data7_1000[start_index:stop_index]
DAY3000_subset_30007

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(1015.405, 1042.36]",2022-06-07 14:38:12.000,2022-06-07 14:38:13.000,1030.120,1490.670,1455.59,1849.82,1634.34,1395.52,981.430,647.840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1042.36, 1069.316]",2022-06-07 14:38:21.000,2022-06-07 14:38:22.000,1055.940,1102.160,1265.07,1690.46,1607.14,1411.47,945.350,758.820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1069.316, 1096.272]",2022-06-07 14:38:32.000,2022-06-07 14:38:33.000,1084.140,962.470,1327.31,1561.55,1793.10,1532.14,907.590,598.810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1096.272, 1123.227]",2022-06-07 14:38:42.500,2022-06-07 14:38:43.500,1107.855,959.755,1229.99,1499.81,1685.87,1290.73,899.715,529.895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1123.227, 1150.183]",2022-06-07 14:38:52.000,2022-06-07 14:38:53.000,1136.620,914.990,1131.11,1498.62,1490.64,1489.20,885.460,570.760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2659.696, 2686.652]",2022-06-07 14:46:31.000,2022-06-07 14:46:32.000,2672.740,1007.230,465.41,230.12,59.67,38.34,55.520,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(2686.652, 2713.608]",2022-06-07 14:46:36.000,2022-06-07 14:46:37.000,2699.640,766.940,430.16,230.08,119.46,57.35,18.550,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(2713.608, 2740.563]",2022-06-07 14:46:41.000,2022-06-07 14:46:42.000,2726.120,936.420,497.73,209.17,89.62,19.15,37.010,27.190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1431]:
LAS7b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220607_RA_L2_20230131T122325.ict')
start_index = np.where(LAS7b["Time_Start"]==pd.to_datetime("2022-06-07 19:08:00"))
stop_index = np.where(LAS7b["Time_Start"]==pd.to_datetime("2022-06-07 19:23:00"))
    
LAS7b_subset = LAS7b[start_index[0][0]:stop_index[0][0]]
LAS7b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10420,2022-06-07 19:08:00,2022-06-07 19:08:01,133.320,328.68,64.26,14.72,1198.03,1259.03,1122.71,954.00,...,0.0,51.79,17.27,0.0,0.0,0.00,10.35,10.35,0.0,1.087
10421,2022-06-07 19:08:01,2022-06-07 19:08:02,133.120,302.80,44.46,7.49,1033.58,1163.43,1064.34,1106.61,...,0.0,0.00,17.32,0.0,0.0,10.38,10.39,0.00,0.0,1.087
10422,2022-06-07 19:08:02,2022-06-07 19:08:03,132.930,312.22,49.98,8.21,987.21,1298.72,1065.12,1018.70,...,0.0,34.68,34.70,0.0,0.0,10.41,10.41,0.00,0.0,1.087
10423,2022-06-07 19:08:03,2022-06-07 19:08:04,132.720,320.26,38.30,5.31,1005.87,1192.24,977.17,923.42,...,0.0,17.23,0.00,0.0,0.0,0.00,10.35,0.00,0.0,1.087
10424,2022-06-07 19:08:04,2022-06-07 19:08:05,132.595,265.97,31.34,4.87,791.47,1192.98,915.60,894.76,...,0.0,0.00,0.00,0.0,0.0,0.00,10.37,0.00,0.0,1.088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11315,2022-06-07 19:22:55,2022-06-07 19:22:56,3518.030,366.87,20.48,0.54,2164.07,1794.33,1358.18,897.29,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,1.541
11316,2022-06-07 19:22:56,2022-06-07 19:22:57,3524.890,408.88,23.49,0.63,2786.04,1791.97,1085.19,746.66,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,1.542
11317,2022-06-07 19:22:57,2022-06-07 19:22:58,3531.840,376.98,20.60,0.51,2717.05,1363.33,1169.40,957.54,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,1.542
11318,2022-06-07 19:22:58,2022-06-07 19:22:59,3538.850,372.76,20.13,0.51,2499.50,1828.34,1230.71,657.79,...,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,1.544


In [1432]:
DAY1000_subset_10007 = DAY1000_subset_10007.reset_index(drop=True)
DAY3000_subset_30007 = DAY3000_subset_30007.reset_index(drop=True)

In [1433]:
SMPS7b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220607_RA_L2_20230131T122819.ict')
start_index = np.where(SMPS7b["Time_Start"]==pd.to_datetime("2022-06-07 19:08:00"))
stop_index = np.where(SMPS7b["Time_Start"]==pd.to_datetime("2022-06-07 19:23:00"))
    
SMPS7b_subset = SMPS7b[start_index[0][0]:stop_index[0][0]]
SMPS7b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10420,2022-06-07 19:08:00,2022-06-07 19:08:01,133.320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10421,2022-06-07 19:08:01,2022-06-07 19:08:02,133.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10422,2022-06-07 19:08:02,2022-06-07 19:08:03,132.930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10423,2022-06-07 19:08:03,2022-06-07 19:08:04,132.720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10424,2022-06-07 19:08:04,2022-06-07 19:08:05,132.595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11315,2022-06-07 19:22:55,2022-06-07 19:22:56,3518.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11316,2022-06-07 19:22:56,2022-06-07 19:22:57,3524.890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11317,2022-06-07 19:22:57,2022-06-07 19:22:58,3531.840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11318,2022-06-07 19:22:58,2022-06-07 19:22:59,3538.850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1434]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS7b_subset.drop(columns=columns_to_remove, inplace=True)
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS7b_subset.drop(columns=columns_to_remove, inplace=True)


In [1435]:
#binning data7b into groups
nbins = 100
bins_1000 = np.linspace(data7b.GPS_Altitude_THORNHILL.min(), data7b.GPS_Altitude_THORNHILL.max(), nbins+1)
data7b['Bins'] = pd.cut(data7b['GPS_Altitude_THORNHILL'], bins_1000)
data7b_1000 = data7b.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data7b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data7b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_10007b = data7b_1000[start_index:stop_index]
DAY1000_subset_10007b


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(132.595, 166.728]",2022-06-07 19:08:09.000,2022-06-07 19:08:10.000,138.9650,1005.870,1156.340,1065.120,950.920,764.590,537.640,324.900,...,1204.80,1648.11,1689.44,1772.91,1557.60,1280.20,1080.90,736.49,1088.08,845.31
"(166.728, 200.861]",2022-06-07 19:08:21.500,2022-06-07 19:08:22.500,183.5400,1046.505,1081.025,1207.720,1074.540,937.865,516.280,258.250,...,1204.80,1648.11,1689.44,1772.91,1557.60,1280.20,1080.90,736.49,1088.08,845.31
"(200.861, 234.994]",2022-06-07 19:08:31.500,2022-06-07 19:08:32.500,217.5125,1093.370,1179.940,1135.200,1080.500,888.695,480.475,285.245,...,1204.80,1648.11,1689.44,1772.91,1557.60,1280.20,1080.90,736.49,1088.08,845.31
"(234.994, 269.127]",2022-06-07 19:08:42.500,2022-06-07 19:08:43.500,251.5325,972.410,1092.890,1103.575,958.460,810.925,537.400,366.810,...,1204.80,1648.11,1689.44,1772.91,1557.60,1280.20,1080.90,736.49,1088.08,845.31
"(269.127, 303.26]",2022-06-07 19:08:51.500,2022-06-07 19:08:52.500,286.8050,840.455,996.335,1055.310,971.455,726.620,472.490,353.135,...,1204.80,1648.11,1689.44,1772.91,1557.60,1280.20,1080.90,736.49,1088.08,845.31
"(303.26, 337.393]",2022-06-07 19:09:01.500,2022-06-07 19:09:02.500,318.2300,672.810,913.305,929.330,984.100,745.390,454.240,352.695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(337.393, 371.526]",2022-06-07 19:09:12.000,2022-06-07 19:09:13.000,354.8600,743.620,731.560,912.880,892.880,766.990,517.550,297.290,...,1080.93,1076.32,1884.23,1621.31,1264.99,996.60,893.87,982.12,751.47,772.77
"(371.526, 405.659]",2022-06-07 19:09:23.500,2022-06-07 19:09:24.500,389.4000,720.590,947.305,979.290,937.245,839.790,509.080,352.965,...,1080.93,1076.32,1884.23,1621.31,1264.99,996.60,893.87,982.12,751.47,772.77
"(405.659, 439.792]",2022-06-07 19:09:35.000,2022-06-07 19:09:36.000,421.5550,718.240,832.690,1039.870,897.790,725.320,479.740,271.580,...,1080.93,1076.32,1884.23,1621.31,1264.99,996.60,893.87,982.12,751.47,772.77


In [1436]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data7b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data7b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_30007b = data7b_1000[start_index:stop_index]
DAY3000_subset_30007b

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(985.921, 1020.054]",2022-06-07 19:12:28.500,2022-06-07 19:12:29.500,1003.240,2958.485,2209.055,1982.965,1675.910,1326.445,730.700,474.735,...,1178.05,1888.95,2906.14,3524.83,3895.55,3968.81,5401.87,5448.40,5452.12,4525.51
"(1020.054, 1054.187]",2022-06-07 19:12:38.500,2022-06-07 19:12:39.500,1036.910,4563.665,3624.060,2830.115,2363.770,1700.185,1018.300,570.305,...,1178.05,1888.95,2906.14,3524.83,3895.55,3968.81,5401.87,5448.40,5452.12,4525.51
"(1054.187, 1088.32]",2022-06-07 19:12:48.000,2022-06-07 19:12:49.000,1072.330,5019.770,3828.620,2928.650,2455.620,1857.530,1107.950,571.390,...,1178.05,1888.95,2906.14,3524.83,3895.55,3968.81,5401.87,5448.40,5452.12,4525.51
"(1088.32, 1122.453]",2022-06-07 19:12:58.500,2022-06-07 19:12:59.500,1104.765,5223.815,3885.925,3253.055,2436.615,1862.205,982.300,624.805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1122.453, 1156.586]",2022-06-07 19:13:10.500,2022-06-07 19:13:11.500,1140.005,3152.365,2455.090,2238.100,1834.680,1446.915,875.225,502.090,...,1258.65,879.37,1308.94,1305.81,1368.94,713.83,1415.42,1319.58,1011.50,1263.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3306.969, 3341.102]",2022-06-07 19:22:24.000,2022-06-07 19:22:25.000,3325.470,1296.400,897.530,669.050,359.070,210.450,166.890,27.130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3341.102, 3375.235]",2022-06-07 19:22:29.500,2022-06-07 19:22:30.500,3359.530,1249.915,979.995,679.205,433.480,315.595,92.710,81.440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3375.235, 3409.368]",2022-06-07 19:22:35.000,2022-06-07 19:22:36.000,3393.950,1486.990,997.320,667.530,418.050,210.320,92.600,135.510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1437]:
DAY1000_subset_10007b = DAY1000_subset_10007b.reset_index(drop=True)
DAY3000_subset_30007b = DAY3000_subset_30007b.reset_index(drop=True)

In [1438]:
LAS8 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220608_RA_L2_20230131T122327.ict')
start_index = np.where(LAS8["Time_Start"]==pd.to_datetime("2022-06-08 20:30:00"))
stop_index = np.where(LAS8["Time_Start"]==pd.to_datetime("2022-06-08 20:45:00"))
    
LAS8_subset = LAS8[start_index[0][0]:stop_index[0][0]]
LAS8_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10668,2022-06-08 20:30:00,2022-06-08 20:30:01,152.725,296.23,76.67,22.79,311.02,662.20,705.20,1279.39,...,0.00,0.00,17.23,12.92,0.0,20.67,0.00,0.0,10.34,1.091
10669,2022-06-08 20:30:01,2022-06-08 20:30:02,153.350,334.25,62.58,18.32,409.05,532.74,1084.41,1078.12,...,0.00,17.33,0.00,0.00,0.0,0.00,0.00,0.0,10.40,1.090
10670,2022-06-08 20:30:02,2022-06-08 20:30:03,153.870,346.35,26.73,0.75,407.92,696.54,937.32,1134.82,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1.091
10671,2022-06-08 20:30:03,2022-06-08 20:30:04,154.255,344.48,37.84,3.30,384.66,532.45,960.06,1406.49,...,0.00,0.00,17.33,0.00,0.0,10.39,0.00,0.0,0.00,1.091
10672,2022-06-08 20:30:04,2022-06-08 20:30:05,154.510,327.79,41.96,5.56,552.87,531.67,959.65,1105.80,...,0.00,34.62,0.00,0.00,0.0,0.00,10.39,0.0,0.00,1.090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11563,2022-06-08 20:44:55,2022-06-08 20:44:56,3838.620,424.27,22.33,0.69,3218.05,2025.71,1356.02,836.31,...,25.91,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1.588
11564,2022-06-08 20:44:56,2022-06-08 20:44:57,3847.170,402.98,20.35,0.49,3029.46,1894.65,1357.58,658.19,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1.589
11565,2022-06-08 20:44:57,2022-06-08 20:44:58,3855.660,430.89,22.17,0.55,3050.43,1925.83,1397.16,985.28,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1.590
11566,2022-06-08 20:44:58,2022-06-08 20:44:59,3864.100,451.64,20.76,0.45,3175.28,2559.83,1589.13,987.43,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1.592


In [1439]:
SMPS8 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220608_RA_L2_20230131T122820.ict')
start_index = np.where(SMPS8["Time_Start"]==pd.to_datetime("2022-06-08 20:30:00"))
stop_index = np.where(SMPS8["Time_Start"]==pd.to_datetime("2022-06-08 20:45:00"))
    
SMPS8_subset = SMPS8[start_index[0][0]:stop_index[0][0]]
SMPS8_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10668,2022-06-08 20:30:00,2022-06-08 20:30:01,152.725,544.29,4.05,0.04,0.0,0.0,0.0,0.0,...,1160.3,1555.18,1663.99,1218.46,939.85,670.09,489.84,270.3,473.68,1.091
10669,2022-06-08 20:30:01,2022-06-08 20:30:02,153.350,544.29,4.05,0.04,0.0,0.0,0.0,0.0,...,1160.3,1555.18,1663.99,1218.46,939.85,670.09,489.84,270.3,473.68,1.091
10670,2022-06-08 20:30:02,2022-06-08 20:30:03,153.870,544.29,4.05,0.04,0.0,0.0,0.0,0.0,...,1160.3,1555.18,1663.99,1218.46,939.85,670.09,489.84,270.3,473.68,1.091
10671,2022-06-08 20:30:03,2022-06-08 20:30:04,154.255,544.29,4.05,0.04,0.0,0.0,0.0,0.0,...,1160.3,1555.18,1663.99,1218.46,939.85,670.09,489.84,270.3,473.68,1.091
10672,2022-06-08 20:30:04,2022-06-08 20:30:05,154.510,544.29,4.05,0.04,0.0,0.0,0.0,0.0,...,1160.3,1555.18,1663.99,1218.46,939.85,670.09,489.84,270.3,473.68,1.091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11563,2022-06-08 20:44:55,2022-06-08 20:44:56,3838.620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.625
11564,2022-06-08 20:44:56,2022-06-08 20:44:57,3847.170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.625
11565,2022-06-08 20:44:57,2022-06-08 20:44:58,3855.660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.625
11566,2022-06-08 20:44:58,2022-06-08 20:44:59,3864.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.625


In [1440]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS8_subset.drop(columns=columns_to_remove, inplace=True)

In [1441]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS8_subset.drop(columns=columns_to_remove, inplace=True)

In [1442]:
data8 = add_sum(LAS8_subset, SMPS8_subset)

In [1443]:
#binning data8 into groups
nbins = 100
bins_1000 = np.linspace(data8.GPS_Altitude_THORNHILL.min(), data8.GPS_Altitude_THORNHILL.max(), nbins+1)
data8['Bins'] = pd.cut(data8['GPS_Altitude_THORNHILL'], bins_1000)
data8_1000 = data8.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data8_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data8_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_10008 = data8_1000[start_index:stop_index]
DAY1000_subset_10008

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(147.455, 184.705]",2022-06-08 20:30:28.000,2022-06-08 20:30:29.000,154.5400,407.920,596.150,835.200,1196.650,1434.390,965.310,516.470,...,1078.63,1160.30,1555.18,1663.99,1218.46,939.85,670.09,489.84,270.30,473.68
"(184.705, 221.955]",2022-06-08 20:31:07.000,2022-06-08 20:31:08.000,198.9850,407.670,663.900,828.340,1162.420,1395.440,888.880,539.150,...,920.58,1047.31,1184.20,1157.94,1150.85,995.93,552.67,486.30,458.77,560.59
"(221.955, 259.205]",2022-06-08 20:31:21.500,2022-06-08 20:31:22.500,242.1600,348.040,446.720,719.430,1062.110,1354.700,959.880,474.070,...,920.58,1047.31,1184.20,1157.94,1150.85,995.93,552.67,486.30,458.77,560.59
"(259.205, 296.455]",2022-06-08 20:31:32.500,2022-06-08 20:31:33.500,278.0025,288.900,481.900,719.090,1105.965,1363.535,1092.870,556.915,...,920.58,1047.31,1184.20,1157.94,1150.85,995.93,552.67,486.30,458.77,560.59
"(296.455, 333.705]",2022-06-08 20:31:44.000,2022-06-08 20:31:45.000,314.4600,337.330,432.360,729.520,1075.170,1319.980,1035.680,596.640,...,920.58,1047.31,1184.20,1157.94,1150.85,995.93,552.67,486.30,458.77,560.59
"(333.705, 370.955]",2022-06-08 20:31:54.500,2022-06-08 20:31:55.500,352.5375,336.250,415.920,749.625,1116.625,1402.735,1027.170,555.150,...,766.40,1224.92,1345.24,1252.39,1054.49,1119.14,656.49,727.65,585.87,526.31
"(370.955, 408.205]",2022-06-08 20:32:04.500,2022-06-08 20:32:05.500,389.1225,277.305,416.540,732.725,1095.245,1364.270,1065.970,569.695,...,766.40,1224.92,1345.24,1252.39,1054.49,1119.14,656.49,727.65,585.87,526.31
"(408.205, 445.455]",2022-06-08 20:32:15.500,2022-06-08 20:32:16.500,425.9075,263.715,479.960,668.705,1047.865,1511.245,1017.415,597.340,...,766.40,1224.92,1345.24,1252.39,1054.49,1119.14,656.49,727.65,585.87,526.31
"(445.455, 482.705]",2022-06-08 20:32:27.000,2022-06-08 20:32:28.000,462.3600,312.610,462.980,747.200,1049.790,1392.050,1074.900,623.330,...,766.40,1224.92,1345.24,1252.39,1054.49,1119.14,656.49,727.65,585.87,526.31


In [1444]:
DAY1000_subset_10008 = DAY1000_subset_10008.reset_index(drop=True)

In [1445]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data8_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data8_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_30008 = data8_1000[start_index:stop_index]
DAY3000_subset_30008

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(1004.206, 1041.456]",2022-06-08 20:35:22.000,2022-06-08 20:35:23.000,1020.120,96.060,132.980,209.290,149.400,190.790,277.420,162.960,...,2908.05,2359.3,3811.32,2596.25,1689.02,902.77,783.13,945.10,520.66,637.18
"(1041.456, 1078.706]",2022-06-08 20:35:35.000,2022-06-08 20:35:36.000,1060.190,215.450,266.130,354.010,627.470,877.150,888.240,488.260,...,2908.05,2359.3,3811.32,2596.25,1689.02,902.77,783.13,945.10,520.66,637.18
"(1078.706, 1115.956]",2022-06-08 20:35:48.500,2022-06-08 20:35:49.500,1096.685,371.945,398.450,416.550,790.240,1259.620,1295.400,839.970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1115.956, 1153.206]",2022-06-08 20:36:00.500,2022-06-08 20:36:01.500,1134.755,228.285,332.480,521.965,688.365,1302.480,1358.985,857.520,...,244.86,1030.7,605.80,695.38,867.68,755.82,442.16,542.02,925.04,685.74
"(1153.206, 1190.456]",2022-06-08 20:36:12.500,2022-06-08 20:36:13.500,1173.080,408.360,415.570,501.370,718.265,1031.960,1056.425,638.290,...,244.86,1030.7,605.80,695.38,867.68,755.82,442.16,542.02,925.04,685.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3276.459, 3313.709]",2022-06-08 20:43:51.000,2022-06-08 20:43:52.000,3293.800,1778.310,1296.610,877.450,628.510,344.520,166.530,135.670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3313.709, 3350.959]",2022-06-08 20:43:55.500,2022-06-08 20:43:56.500,3332.045,2161.010,1362.195,855.860,582.805,439.925,250.065,94.980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3350.959, 3388.209]",2022-06-08 20:44:00.000,2022-06-08 20:44:01.000,3370.540,2068.530,1560.870,939.780,658.110,420.700,222.450,190.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1446]:
DAY3000_subset_30008 = DAY3000_subset_30008.reset_index(drop=True)

In [1447]:
LAS10 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220610_RA_L1_20230131T122328.ict')
start_index = np.where(LAS10["Time_Start"]==pd.to_datetime("2022-06-10 15:17:00"))
stop_index = np.where(LAS10["Time_Start"]==pd.to_datetime("2022-06-10 15:25:00"))
    
LAS10_subset = LAS10[start_index[0][0]:stop_index[0][0]]
LAS10_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10616,2022-06-10 15:17:00,2022-06-10 15:17:01,122.040,188.02,25.75,1.32,95.98,132.51,229.16,656.42,...,0.00,0.00,0.0,12.96,0.00,0.0,0.00,0.0,0.00,1.095
10617,2022-06-10 15:17:01,2022-06-10 15:17:02,122.105,152.75,24.45,1.74,216.28,166.18,208.19,358.72,...,0.00,34.62,0.0,25.96,0.00,0.0,0.00,0.0,0.00,1.095
10618,2022-06-10 15:17:02,2022-06-10 15:17:03,122.170,123.10,38.54,6.92,144.34,132.63,187.32,179.37,...,0.00,0.00,52.0,0.00,10.39,0.0,10.39,0.0,0.00,1.095
10619,2022-06-10 15:17:03,2022-06-10 15:17:04,122.240,157.26,31.50,2.87,216.67,66.75,271.75,449.25,...,51.96,17.33,0.0,13.01,10.41,0.0,0.00,0.0,0.00,1.095
10620,2022-06-10 15:17:04,2022-06-10 15:17:05,122.315,182.92,71.07,20.52,241.08,266.84,293.02,300.11,...,0.00,52.10,69.5,13.02,0.00,0.0,0.00,0.0,10.42,1.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11091,2022-06-10 15:24:55,2022-06-10 15:24:56,3540.260,196.78,12.13,0.40,1417.75,863.18,542.18,298.95,...,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,1.549
11092,2022-06-10 15:24:56,2022-06-10 15:24:57,3546.970,204.61,10.58,0.26,1274.86,963.97,792.94,538.26,...,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,1.550
11093,2022-06-10 15:24:57,2022-06-10 15:24:58,3553.620,236.45,12.84,0.35,1853.48,1163.62,669.05,419.49,...,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,1.551
11094,2022-06-10 15:24:58,2022-06-10 15:24:59,3560.200,189.53,11.63,0.48,1417.25,763.43,688.77,328.62,...,0.00,17.30,0.0,0.00,0.00,0.0,0.00,0.0,0.00,1.552


In [1448]:
SMPS10 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220610_RA_L1_20230131T122821.ict')
start_index = np.where(SMPS10["Time_Start"]==pd.to_datetime("2022-06-10 15:17:00"))
stop_index = np.where(SMPS10["Time_Start"]==pd.to_datetime("2022-06-10 15:25:00"))
    
SMPS10_subset = SMPS10[start_index[0][0]:stop_index[0][0]]
SMPS10_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10616,2022-06-10 15:17:00,2022-06-10 15:17:01,122.040,266.94,1.9,0.02,0.0,0.0,0.0,0.0,...,306.29,440.89,322.11,614.07,812.49,519.63,197.71,53.83,211.18,1.095
10617,2022-06-10 15:17:01,2022-06-10 15:17:02,122.105,266.94,1.9,0.02,0.0,0.0,0.0,0.0,...,306.29,440.89,322.11,614.07,812.49,519.63,197.71,53.83,211.18,1.095
10618,2022-06-10 15:17:02,2022-06-10 15:17:03,122.170,266.94,1.9,0.02,0.0,0.0,0.0,0.0,...,306.29,440.89,322.11,614.07,812.49,519.63,197.71,53.83,211.18,1.095
10619,2022-06-10 15:17:03,2022-06-10 15:17:04,122.240,266.94,1.9,0.02,0.0,0.0,0.0,0.0,...,306.29,440.89,322.11,614.07,812.49,519.63,197.71,53.83,211.18,1.095
10620,2022-06-10 15:17:04,2022-06-10 15:17:05,122.315,266.94,1.9,0.02,0.0,0.0,0.0,0.0,...,306.29,440.89,322.11,614.07,812.49,519.63,197.71,53.83,211.18,1.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11091,2022-06-10 15:24:55,2022-06-10 15:24:56,3540.260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.566
11092,2022-06-10 15:24:56,2022-06-10 15:24:57,3546.970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.566
11093,2022-06-10 15:24:57,2022-06-10 15:24:58,3553.620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.566
11094,2022-06-10 15:24:58,2022-06-10 15:24:59,3560.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.566


In [1449]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS10_subset.drop(columns=columns_to_remove, inplace=True)

In [1450]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS10_subset.drop(columns=columns_to_remove, inplace=True)

In [1451]:
data10 = add_sum(LAS10_subset, SMPS10_subset)

In [1452]:
#binning data10 into groups
nbins = 100
bins_1000 = np.linspace(data10.GPS_Altitude_THORNHILL.min(), data10.GPS_Altitude_THORNHILL.max(), nbins+1)
data10['Bins'] = pd.cut(data10['GPS_Altitude_THORNHILL'], bins_1000)
data10_1000 = data10.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data10_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data10_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100010 = data10_1000[start_index:stop_index]
DAY1000_subset_100010


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(122.04, 156.487]",2022-06-10 15:17:25.000,2022-06-10 15:17:26.000,123.7050,192.300,199.230,249.770,327.140,381.050,461.730,408.660,...,375.00,306.29,440.89,322.11,614.07,812.49,519.63,197.71,53.83,211.18
"(156.487, 190.934]",2022-06-10 15:17:52.500,2022-06-10 15:17:53.500,173.7850,144.325,282.870,240.000,254.660,391.300,361.415,406.960,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
"(190.934, 225.381]",2022-06-10 15:17:58.000,2022-06-10 15:17:59.000,209.2900,215.620,230.760,272.220,238.780,362.540,368.350,433.770,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
"(225.381, 259.828]",2022-06-10 15:18:03.000,2022-06-10 15:18:04.000,244.1150,192.080,199.300,208.530,239.400,305.130,407.690,379.780,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
"(259.828, 294.274]",2022-06-10 15:18:07.500,2022-06-10 15:18:08.500,277.9725,96.000,165.715,197.485,194.450,313.935,489.690,365.755,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
"(294.274, 328.721]",2022-06-10 15:18:12.000,2022-06-10 15:18:13.000,312.6600,95.970,199.130,187.280,239.680,362.120,388.590,406.840,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
"(328.721, 363.168]",2022-06-10 15:18:16.500,2022-06-10 15:18:17.500,349.4500,132.055,199.280,177.655,149.690,354.410,406.990,556.170,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
"(363.168, 397.615]",2022-06-10 15:18:20.500,2022-06-10 15:18:21.500,383.6350,144.180,166.140,155.965,163.945,296.135,388.535,421.200,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
"(397.615, 432.062]",2022-06-10 15:18:24.500,2022-06-10 15:18:25.500,417.9100,84.200,99.965,177.720,299.430,325.355,425.585,474.875,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66


In [1453]:
DAY1000_subset_100010 = DAY1000_subset_100010.reset_index(drop=True)

In [1454]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data10_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data10_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300010 = data10_1000[start_index:stop_index]
DAY3000_subset_300010

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(983.212, 1017.659]",2022-06-10 15:19:33.500,2022-06-10 15:19:34.500,1002.010,588.130,647.275,740.280,611.740,496.675,629.415,515.535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1017.659, 1052.106]",2022-06-10 15:19:37.500,2022-06-10 15:19:38.500,1036.915,782.030,798.865,636.765,734.140,554.875,669.350,462.375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1052.106, 1086.553]",2022-06-10 15:19:41.500,2022-06-10 15:19:42.500,1072.325,660.620,714.505,679.205,732.705,610.305,500.380,353.310,...,956.23,963.66,1070.83,1064.71,718.07,913.82,659.86,552.77,1091.01,952.28
"(1086.553, 1121.0]",2022-06-10 15:19:45.000,2022-06-10 15:19:46.000,1103.920,745.140,730.280,751.620,686.920,669.280,666.720,407.330,...,956.23,963.66,1070.83,1064.71,718.07,913.82,659.86,552.77,1091.01,952.28
"(1121.0, 1155.447]",2022-06-10 15:19:48.500,2022-06-10 15:19:49.500,1135.630,636.365,713.835,769.480,776.345,781.470,601.035,530.530,...,956.23,963.66,1070.83,1064.71,718.07,913.82,659.86,552.77,1091.01,952.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3291.155, 3325.602]",2022-06-10 15:24:24.000,2022-06-10 15:24:25.000,3308.150,913.180,697.920,522.540,329.180,210.190,129.760,54.210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3325.602, 3360.049]",2022-06-10 15:24:29.000,2022-06-10 15:24:30.000,3344.300,1153.290,663.070,438.250,329.340,210.290,110.900,54.240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3360.049, 3394.496]",2022-06-10 15:24:33.500,2022-06-10 15:24:34.500,3377.515,1057.755,747.500,563.130,313.940,200.905,101.925,81.460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1455]:
DAY3000_subset_300010 = DAY3000_subset_300010.reset_index(drop=True)

In [1456]:
LAS1110b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220610_RA_L2_20230131T122328.ict')
start_index = np.where(LAS1110b["Time_Start"]==pd.to_datetime("2022-06-10 19:30:00"))
stop_index = np.where(LAS1110b["Time_Start"]==pd.to_datetime("2022-06-10 19:51:00"))
    
LAS1110b_subset = LAS1110b[start_index[0][0]:stop_index[0][0]]
LAS1110b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
7182,2022-06-10 19:30:00,2022-06-10 19:30:01,113.375,272.40,40.20,2.55,431.93,365.07,686.90,626.82,...,0.00,17.28,17.28,25.92,0.00,0.0,0.00,0.00,0.00,1.099
7183,2022-06-10 19:30:01,2022-06-10 19:30:02,113.225,260.92,71.27,15.85,649.50,465.40,459.70,538.54,...,0.00,0.00,17.34,12.99,10.39,0.0,10.40,10.39,0.00,1.099
7184,2022-06-10 19:30:02,2022-06-10 19:30:03,113.065,250.78,43.88,3.34,577.10,398.73,438.04,448.66,...,25.98,17.32,51.99,25.98,0.00,0.0,0.00,0.00,0.00,1.099
7185,2022-06-10 19:30:03,2022-06-10 19:30:04,112.970,236.54,78.48,20.03,335.39,463.74,520.76,416.89,...,0.00,17.26,17.26,0.00,10.35,0.0,0.00,20.70,0.00,1.099
7186,2022-06-10 19:30:04,2022-06-10 19:30:05,112.985,245.75,77.74,23.07,359.93,397.88,583.10,328.57,...,0.00,34.56,0.00,0.00,0.00,0.0,10.37,0.00,10.37,1.099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8437,2022-06-10 19:50:55,2022-06-10 19:50:56,4967.450,421.25,19.48,0.42,3335.21,2022.65,1313.25,865.19,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,1.789
8438,2022-06-10 19:50:56,2022-06-10 19:50:57,4964.110,436.60,23.63,0.84,3195.42,2292.37,1463.05,747.03,...,0.00,0.00,17.30,0.00,0.00,0.0,0.00,0.00,0.00,1.787
8439,2022-06-10 19:50:57,2022-06-10 19:50:58,4960.810,428.02,22.54,0.66,3454.18,2121.47,1333.82,626.48,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,1.788
8440,2022-06-10 19:50:58,2022-06-10 19:50:59,4957.520,391.70,20.16,0.56,2883.56,2058.74,1085.05,1105.33,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,1.788


In [1457]:
SMPS1110b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220610_RA_L2_20230131T122821.ict')
start_index = np.where(SMPS1110b["Time_Start"]==pd.to_datetime("2022-06-10 19:30:00"))
stop_index = np.where(SMPS1110b["Time_Start"]==pd.to_datetime("2022-06-10 19:51:00"))
    
SMPS1110b_subset = SMPS1110b[start_index[0][0]:stop_index[0][0]]
SMPS1110b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
7182,2022-06-10 19:30:00,2022-06-10 19:30:01,113.375,366.64,3.87,0.04,0.0,0.0,0.0,0.00,...,217.59,974.25,1034.42,838.48,836.94,726.33,852.75,566.26,607.64,1.100
7183,2022-06-10 19:30:01,2022-06-10 19:30:02,113.225,366.64,3.87,0.04,0.0,0.0,0.0,0.00,...,217.59,974.25,1034.42,838.48,836.94,726.33,852.75,566.26,607.64,1.100
7184,2022-06-10 19:30:02,2022-06-10 19:30:03,113.065,366.64,3.87,0.04,0.0,0.0,0.0,0.00,...,217.59,974.25,1034.42,838.48,836.94,726.33,852.75,566.26,607.64,1.100
7185,2022-06-10 19:30:03,2022-06-10 19:30:04,112.970,366.64,3.87,0.04,0.0,0.0,0.0,0.00,...,217.59,974.25,1034.42,838.48,836.94,726.33,852.75,566.26,607.64,1.100
7186,2022-06-10 19:30:04,2022-06-10 19:30:05,112.985,366.64,3.87,0.04,0.0,0.0,0.0,0.00,...,217.59,974.25,1034.42,838.48,836.94,726.33,852.75,566.26,607.64,1.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8437,2022-06-10 19:50:55,2022-06-10 19:50:56,4967.450,3678.74,16.84,0.16,NaN,NaN,NaN,4381.73,...,4966.54,4690.58,4207.90,3266.06,3234.08,2605.39,2355.98,2168.97,1714.54,1.787
8438,2022-06-10 19:50:56,2022-06-10 19:50:57,4964.110,3678.74,16.84,0.16,NaN,NaN,NaN,4381.73,...,4966.54,4690.58,4207.90,3266.06,3234.08,2605.39,2355.98,2168.97,1714.54,1.787
8439,2022-06-10 19:50:57,2022-06-10 19:50:58,4960.810,3678.74,16.84,0.16,NaN,NaN,NaN,4381.73,...,4966.54,4690.58,4207.90,3266.06,3234.08,2605.39,2355.98,2168.97,1714.54,1.787
8440,2022-06-10 19:50:58,2022-06-10 19:50:59,4957.520,3678.74,16.84,0.16,NaN,NaN,NaN,4381.73,...,4966.54,4690.58,4207.90,3266.06,3234.08,2605.39,2355.98,2168.97,1714.54,1.787


In [1458]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS1110b_subset.drop(columns=columns_to_remove, inplace=True)

In [1459]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS1110b_subset.drop(columns=columns_to_remove, inplace=True)

In [1460]:
data1110b = add_sum(LAS1110b_subset, SMPS1110b_subset)

In [1461]:
#binning data1110b into groups
nbins = 100
bins_1000 = np.linspace(data1110b.GPS_Altitude_THORNHILL.min(), data1110b.GPS_Altitude_THORNHILL.max(), nbins+1)
data1110b['Bins'] = pd.cut(data1110b['GPS_Altitude_THORNHILL'], bins_1000)
data1110b_1000 = data1110b.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data1110b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data1110b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_10001110b = data1110b_1000[start_index:stop_index]
DAY1000_subset_10001110b


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(112.97, 161.788]",2022-06-10 19:32:19.500,2022-06-10 19:32:20.500,115.2325,481.010,400.005,418.950,476.800,573.270,648.970,664.650,...,170.68,441.71,833.91,971.38,838.48,931.86,920.94,1041.72,1037.72,739.42
"(161.788, 210.605]",2022-06-10 19:34:43.500,2022-06-10 19:34:44.500,181.9325,528.320,315.210,427.180,389.265,564.160,583.135,652.625,...,368.14,397.27,561.03,741.66,797.02,1456.47,1072.84,778.48,995.21,536.56
"(210.605, 259.422]",2022-06-10 19:34:52.000,2022-06-10 19:34:53.000,232.4700,528.650,431.560,458.260,448.300,552.380,629.330,595.650,...,368.14,397.27,561.03,741.66,797.02,1456.47,1072.84,778.48,995.21,536.56
"(259.422, 308.24]",2022-06-10 19:35:00.000,2022-06-10 19:35:01.000,286.4950,433.160,500.100,436.830,506.370,612.660,593.390,570.080,...,368.14,397.27,561.03,741.66,797.02,1456.47,1072.84,778.48,995.21,536.56
"(308.24, 357.058]",2022-06-10 19:35:09.000,2022-06-10 19:35:10.000,332.9850,408.960,366.040,354.240,479.580,572.940,666.010,652.280,...,368.14,397.27,561.03,741.66,797.02,1456.47,1072.84,778.48,995.21,536.56
"(357.058, 405.875]",2022-06-10 19:35:18.000,2022-06-10 19:35:19.000,381.7600,479.800,430.910,457.010,448.980,551.090,611.440,677.650,...,368.14,397.27,561.03,741.66,797.02,1456.47,1072.84,778.48,995.21,536.56
"(405.875, 454.692]",2022-06-10 19:35:27.000,2022-06-10 19:35:28.000,429.9200,410.110,430.060,460.260,510.190,478.950,519.840,625.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(454.692, 503.51]",2022-06-10 19:35:36.000,2022-06-10 19:35:37.000,479.5150,505.220,598.030,500.410,627.330,610.670,684.950,594.800,...,310.39,848.03,700.79,325.60,683.11,1089.03,664.38,527.21,412.82,768.59
"(503.51, 552.328]",2022-06-10 19:35:45.000,2022-06-10 19:35:46.000,529.9600,552.780,597.260,501.410,565.850,709.100,572.010,680.520,...,310.39,848.03,700.79,325.60,683.11,1089.03,664.38,527.21,412.82,768.59


In [1462]:
DAY1000_subset_10001110b = DAY1000_subset_10001110b.reset_index(drop=True)

In [1463]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data1110b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data1110b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_30001110b = data1110b_1000[start_index:stop_index]
DAY3000_subset_30001110b

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(991.685, 1040.502]",2022-06-10 19:37:18.500,2022-06-10 19:37:19.500,1016.095,600.450,697.655,625.955,613.195,716.250,583.490,583.690,...,868.10,915.00,1094.78,1507.59,859.84,1013.18,867.97,736.82,400.75,659.41
"(1040.502, 1089.32]",2022-06-10 19:37:27.500,2022-06-10 19:37:28.500,1061.385,613.760,597.550,510.130,582.260,591.520,573.100,487.450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1089.32, 1138.138]",2022-06-10 19:37:36.000,2022-06-10 19:37:37.000,1116.890,597.690,795.730,813.130,715.770,686.720,536.300,569.090,...,658.57,835.71,867.62,747.77,1256.37,389.81,803.38,1089.94,952.68,1020.49
"(1138.138, 1186.955]",2022-06-10 19:37:45.500,2022-06-10 19:37:46.500,1157.980,840.205,797.690,887.275,701.650,793.700,628.840,501.475,...,658.57,835.71,867.62,747.77,1256.37,389.81,803.38,1089.94,952.68,1020.49
"(1186.955, 1235.772]",2022-06-10 19:37:54.500,2022-06-10 19:37:55.500,1210.350,889.355,812.620,812.700,670.755,689.080,619.825,542.375,...,658.57,835.71,867.62,747.77,1256.37,389.81,803.38,1089.94,952.68,1020.49
"(1235.772, 1284.59]",2022-06-10 19:38:04.500,2022-06-10 19:38:05.500,1261.140,865.035,846.795,794.260,851.380,753.570,704.750,664.165,...,658.57,835.71,867.62,747.77,1256.37,389.81,803.38,1089.94,952.68,1020.49
"(1284.59, 1333.408]",2022-06-10 19:38:14.500,2022-06-10 19:38:15.500,1308.955,816.465,746.930,813.385,747.320,800.120,728.150,584.145,...,658.57,835.71,867.62,747.77,1256.37,389.81,803.38,1089.94,952.68,1020.49
"(1333.408, 1382.225]",2022-06-10 19:38:22.500,2022-06-10 19:38:23.500,1359.380,853.500,780.410,855.610,747.605,765.515,649.070,556.825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1382.225, 1431.042]",2022-06-10 19:38:31.000,2022-06-10 19:38:32.000,1405.730,768.390,730.290,857.340,656.650,668.150,650.450,542.540,...,330.62,797.63,985.88,646.54,520.89,709.99,622.69,676.95,703.90,663.26


In [1464]:
DAY3000_subset_30001110b = DAY3000_subset_30001110b.reset_index(drop=True)

In [1465]:
LAS10b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220610_RA_L2_20230131T122328.ict')
start_index = np.where(LAS10b["Time_Start"]==pd.to_datetime("2022-06-10 20:25:00"))
stop_index = np.where(LAS10b["Time_Start"]==pd.to_datetime("2022-06-10 20:35:00"))
    
LAS10b_subset = LAS10b[start_index[0][0]:stop_index[0][0]]
LAS10b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10482,2022-06-10 20:25:00,2022-06-10 20:25:01,133.810,339.63,64.61,10.52,940.18,932.88,544.25,899.96,...,0.00,17.37,17.36,0.00,0.0,0.00,20.84,0.00,0.00,1.102
10483,2022-06-10 20:25:01,2022-06-10 20:25:02,133.615,316.72,111.30,32.99,933.59,661.85,540.78,476.34,...,51.65,51.68,34.49,0.00,0.0,0.00,10.35,10.34,10.34,1.102
10484,2022-06-10 20:25:02,2022-06-10 20:25:03,133.015,307.95,51.92,5.05,675.45,700.52,565.51,809.58,...,52.09,34.74,17.38,13.03,0.0,10.42,0.00,0.00,0.00,1.103
10485,2022-06-10 20:25:03,2022-06-10 20:25:04,131.855,286.66,51.99,5.28,552.71,564.50,563.92,627.47,...,0.00,17.31,51.91,25.96,0.0,10.38,0.00,0.00,0.00,1.100
10486,2022-06-10 20:25:04,2022-06-10 20:25:05,130.265,288.17,45.96,4.44,528.23,431.42,666.10,747.26,...,25.91,0.00,34.58,12.97,0.0,10.37,0.00,0.00,0.00,1.101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,2022-06-10 20:34:55,2022-06-10 20:34:56,4924.170,211.89,12.55,0.50,1683.35,997.27,543.25,478.22,...,25.94,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.780
11078,2022-06-10 20:34:56,2022-06-10 20:34:57,4924.550,219.32,11.50,0.27,1465.80,930.29,793.92,388.41,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.780
11079,2022-06-10 20:34:57,2022-06-10 20:34:58,4924.900,201.46,11.80,0.32,1153.81,1196.85,417.75,448.51,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.780
11080,2022-06-10 20:34:58,2022-06-10 20:34:59,4925.210,185.23,10.01,0.25,1297.66,730.52,542.55,537.85,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.780


In [1466]:
SMPS10b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220610_RA_L2_20230131T122821.ict')
start_index = np.where(SMPS10b["Time_Start"]==pd.to_datetime("2022-06-10 20:25:00"))
stop_index = np.where(SMPS10b["Time_Start"]==pd.to_datetime("2022-06-10 20:35:00"))
    
SMPS10b_subset = SMPS10b[start_index[0][0]:stop_index[0][0]]
SMPS10b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10482,2022-06-10 20:25:00,2022-06-10 20:25:01,133.810,3119.92,6.91,0.05,0.0,0.0,0.0,0.0,...,1684.36,1477.32,1509.54,1323.70,896.21,883.71,418.90,714.26,396.26,1.102
10483,2022-06-10 20:25:01,2022-06-10 20:25:02,133.615,3119.92,6.91,0.05,0.0,0.0,0.0,0.0,...,1684.36,1477.32,1509.54,1323.70,896.21,883.71,418.90,714.26,396.26,1.102
10484,2022-06-10 20:25:02,2022-06-10 20:25:03,133.015,3119.92,6.91,0.05,0.0,0.0,0.0,0.0,...,1684.36,1477.32,1509.54,1323.70,896.21,883.71,418.90,714.26,396.26,1.102
10485,2022-06-10 20:25:03,2022-06-10 20:25:04,131.855,3119.92,6.91,0.05,0.0,0.0,0.0,0.0,...,1684.36,1477.32,1509.54,1323.70,896.21,883.71,418.90,714.26,396.26,1.102
10486,2022-06-10 20:25:04,2022-06-10 20:25:05,130.265,3119.92,6.91,0.05,0.0,0.0,0.0,0.0,...,1684.36,1477.32,1509.54,1323.70,896.21,883.71,418.90,714.26,396.26,1.102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,2022-06-10 20:34:55,2022-06-10 20:34:56,4924.170,1862.02,8.53,0.08,NaN,NaN,NaN,0.0,...,3307.37,2501.36,3039.15,1740.11,1191.36,1086.99,850.01,843.63,871.88,1.780
11078,2022-06-10 20:34:56,2022-06-10 20:34:57,4924.550,1862.02,8.53,0.08,NaN,NaN,NaN,0.0,...,3307.37,2501.36,3039.15,1740.11,1191.36,1086.99,850.01,843.63,871.88,1.780
11079,2022-06-10 20:34:57,2022-06-10 20:34:58,4924.900,1862.02,8.53,0.08,NaN,NaN,NaN,0.0,...,3307.37,2501.36,3039.15,1740.11,1191.36,1086.99,850.01,843.63,871.88,1.780
11080,2022-06-10 20:34:58,2022-06-10 20:34:59,4925.210,1862.02,8.53,0.08,NaN,NaN,NaN,0.0,...,3307.37,2501.36,3039.15,1740.11,1191.36,1086.99,850.01,843.63,871.88,1.780


In [1467]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS10b_subset.drop(columns=columns_to_remove, inplace=True)

In [1468]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS10b_subset.drop(columns=columns_to_remove, inplace=True)

In [1469]:
data10b = add_sum(LAS10b_subset, SMPS10b_subset)

In [1470]:
#binning data10b into groups
nbins = 100
bins_1000 = np.linspace(data10b.GPS_Altitude_THORNHILL.min(), data10b.GPS_Altitude_THORNHILL.max(), nbins+1)
data10b['Bins'] = pd.cut(data10b['GPS_Altitude_THORNHILL'], bins_1000)
data10b_1000 = data10b.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data10b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data10b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100010b = data10b_1000[start_index:stop_index]
DAY1000_subset_100010b


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(119.575, 167.73]",2022-06-10 20:25:54.000,2022-06-10 20:25:55.000,127.6650,626.640,533.320,498.640,536.470,612.990,594.970,599.220,...,173.66,725.82,856.40,924.18,1005.66,1297.80,1048.14,1317.81,1570.10,1005.83
"(167.73, 215.886]",2022-06-10 20:26:49.500,2022-06-10 20:26:50.500,192.9050,553.820,266.250,428.530,538.820,467.350,621.785,461.150,...,359.59,561.26,943.00,860.01,1100.43,830.19,616.07,917.26,1040.79,428.56
"(215.886, 264.041]",2022-06-10 20:26:53.500,2022-06-10 20:26:54.500,242.3875,575.365,397.525,323.735,432.165,400.480,584.865,623.210,...,359.59,561.26,943.00,860.01,1100.43,830.19,616.07,917.26,1040.79,428.56
"(264.041, 312.196]",2022-06-10 20:26:57.500,2022-06-10 20:26:58.500,289.1125,431.615,348.810,468.665,373.525,544.015,658.400,624.055,...,359.59,561.26,943.00,860.01,1100.43,830.19,616.07,917.26,1040.79,428.56
"(312.196, 360.352]",2022-06-10 20:27:02.000,2022-06-10 20:27:03.000,334.1400,458.530,331.930,354.510,360.220,515.170,539.250,596.880,...,359.59,561.26,943.00,860.01,1100.43,830.19,616.07,917.26,1040.79,428.56
"(360.352, 408.507]",2022-06-10 20:27:07.000,2022-06-10 20:27:08.000,384.2600,385.030,332.510,333.940,387.360,515.540,536.260,570.930,...,359.59,561.26,943.00,860.01,1100.43,830.19,616.07,917.26,1040.79,428.56
"(408.507, 456.662]",2022-06-10 20:27:11.500,2022-06-10 20:27:12.500,435.1125,348.255,431.100,376.230,418.060,468.010,591.080,500.920,...,359.59,561.26,943.00,860.01,1100.43,830.19,616.07,917.26,1040.79,428.56
"(456.662, 504.818]",2022-06-10 20:27:15.000,2022-06-10 20:27:16.000,479.9450,359.640,398.590,395.680,477.050,573.440,462.850,543.000,...,359.59,561.26,943.00,860.01,1100.43,830.19,616.07,917.26,1040.79,428.56
"(504.818, 552.973]",2022-06-10 20:27:18.500,2022-06-10 20:27:19.500,525.2150,493.335,381.380,417.035,477.790,526.255,584.235,557.415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1471]:
DAY1000_subset_100010b = DAY1000_subset_100010b.reset_index(drop=True)

In [1472]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data10b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data10b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300010b = data10b_1000[start_index:stop_index]
DAY3000_subset_300010b

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(986.371, 1034.527]",2022-06-10 20:27:53.000,2022-06-10 20:27:54.000,1010.110,359.480,365.640,333.630,357.770,478.280,463.130,406.190,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1034.527, 1082.682]",2022-06-10 20:27:56.000,2022-06-10 20:27:57.000,1056.050,409.510,399.470,333.450,328.770,401.920,389.470,380.670,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1082.682, 1130.837]",2022-06-10 20:27:59.000,2022-06-10 20:28:00.000,1102.340,553.740,497.800,459.330,387.880,382.200,444.300,488.250,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1130.837, 1178.993]",2022-06-10 20:28:02.500,2022-06-10 20:28:03.500,1156.145,480.635,365.360,354.230,388.520,439.440,388.880,366.425,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1178.993, 1227.148]",2022-06-10 20:28:06.000,2022-06-10 20:28:07.000,1207.910,383.650,496.930,395.900,357.660,496.490,407.200,379.270,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1227.148, 1275.303]",2022-06-10 20:28:09.000,2022-06-10 20:28:10.000,1253.350,432.400,132.810,375.470,478.090,400.300,408.300,352.830,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1275.303, 1323.459]",2022-06-10 20:28:12.000,2022-06-10 20:28:13.000,1302.510,504.080,398.350,479.350,358.760,420.290,445.260,406.970,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1323.459, 1371.614]",2022-06-10 20:28:15.000,2022-06-10 20:28:16.000,1353.630,433.690,532.580,396.490,597.760,554.320,333.870,380.780,...,416.90,270.89,365.62,663.26,553.71,291.98,879.58,568.36,531.31,398.26
"(1371.614, 1419.769]",2022-06-10 20:28:17.500,2022-06-10 20:28:18.500,1395.860,504.115,663.455,499.925,492.880,467.960,508.730,366.195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1473]:
DAY3000_subset_3000a = DAY3000_subset_3000a.reset_index(drop=True)

In [1474]:
LAS11 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220611_RA_L1_20230131T122329.ict')
start_index = np.where(LAS11["Time_Start"]==pd.to_datetime("2022-06-11 15:24:00"))
stop_index = np.where(LAS11["Time_Start"]==pd.to_datetime("2022-06-11 15:42:00"))
    
LAS11_subset = LAS11[start_index[0][0]:stop_index[0][0]]
LAS11_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10800,2022-06-11 15:24:00,2022-06-11 15:24:01,137.985,225.29,52.01,8.09,263.53,364.63,457.23,447.21,...,0.00,17.26,17.26,0.00,0.00,10.35,10.35,0.00,0.0,1.096
10801,2022-06-11 15:24:01,2022-06-11 15:24:02,138.245,224.89,76.39,18.50,433.61,565.98,230.39,329.55,...,0.00,0.00,17.34,0.00,0.00,20.82,10.41,10.41,0.0,1.096
10802,2022-06-11 15:24:02,2022-06-11 15:24:03,138.485,214.03,51.15,7.38,432.22,233.00,312.79,239.00,...,0.00,17.30,17.31,12.97,10.37,0.00,10.38,0.00,0.0,1.096
10803,2022-06-11 15:24:03,2022-06-11 15:24:04,138.740,243.36,39.81,2.96,289.57,400.39,251.21,480.35,...,0.00,34.74,17.37,0.00,10.42,0.00,0.00,0.00,0.0,1.096
10804,2022-06-11 15:24:04,2022-06-11 15:24:05,139.035,227.53,81.97,16.91,457.11,366.09,251.02,239.43,...,25.99,17.33,51.97,12.99,10.39,20.78,0.00,10.39,0.0,1.096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,2022-06-11 15:41:55,2022-06-11 15:41:56,4701.700,581.87,27.91,0.63,4307.12,2960.38,1715.58,1017.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,1.730
11876,2022-06-11 15:41:56,2022-06-11 15:41:57,4708.440,537.41,26.19,0.61,3841.25,2920.30,1585.42,1164.83,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,1.732
11877,2022-06-11 15:41:57,2022-06-11 15:41:58,4714.800,471.79,22.76,0.52,3606.58,2426.28,1419.26,807.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,1.733
11878,2022-06-11 15:41:58,2022-06-11 15:41:59,4720.960,476.99,24.40,0.63,3389.10,2392.27,1607.41,956.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,1.735


In [1475]:
SMPS11 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220611_RA_L1_20230131T122822.ict')
start_index = np.where(SMPS11["Time_Start"]==pd.to_datetime("2022-06-11 15:24:00"))
stop_index = np.where(SMPS11["Time_Start"]==pd.to_datetime("2022-06-11 15:42:00"))
    
SMPS11_subset = SMPS11[start_index[0][0]:stop_index[0][0]]
SMPS11_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10800,2022-06-11 15:24:00,2022-06-11 15:24:01,137.985,384.27,3.83,0.04,0.0,0.0,0.0,0.0,...,199.91,762.62,857.45,1055.11,1483.20,880.79,875.62,431.06,315.76,1.096
10801,2022-06-11 15:24:01,2022-06-11 15:24:02,138.245,384.27,3.83,0.04,0.0,0.0,0.0,0.0,...,199.91,762.62,857.45,1055.11,1483.20,880.79,875.62,431.06,315.76,1.096
10802,2022-06-11 15:24:02,2022-06-11 15:24:03,138.485,384.27,3.83,0.04,0.0,0.0,0.0,0.0,...,199.91,762.62,857.45,1055.11,1483.20,880.79,875.62,431.06,315.76,1.096
10803,2022-06-11 15:24:03,2022-06-11 15:24:04,138.740,384.27,3.83,0.04,0.0,0.0,0.0,0.0,...,199.91,762.62,857.45,1055.11,1483.20,880.79,875.62,431.06,315.76,1.096
10804,2022-06-11 15:24:04,2022-06-11 15:24:05,139.035,384.27,3.83,0.04,0.0,0.0,0.0,0.0,...,199.91,762.62,857.45,1055.11,1483.20,880.79,875.62,431.06,315.76,1.096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,2022-06-11 15:41:55,2022-06-11 15:41:56,4701.700,4492.17,20.59,0.20,NaN,NaN,NaN,0.0,...,5886.08,6154.82,4304.46,5146.92,3786.81,3637.93,2643.29,2687.39,2353.27,1.741
11876,2022-06-11 15:41:56,2022-06-11 15:41:57,4708.440,4492.17,20.59,0.20,NaN,NaN,NaN,0.0,...,5886.08,6154.82,4304.46,5146.92,3786.81,3637.93,2643.29,2687.39,2353.27,1.741
11877,2022-06-11 15:41:57,2022-06-11 15:41:58,4714.800,4492.17,20.59,0.20,NaN,NaN,NaN,0.0,...,5886.08,6154.82,4304.46,5146.92,3786.81,3637.93,2643.29,2687.39,2353.27,1.741
11878,2022-06-11 15:41:58,2022-06-11 15:41:59,4720.960,4492.17,20.59,0.20,NaN,NaN,NaN,0.0,...,5886.08,6154.82,4304.46,5146.92,3786.81,3637.93,2643.29,2687.39,2353.27,1.741


In [1476]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS11_subset.drop(columns=columns_to_remove, inplace=True)

In [1477]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS11_subset.drop(columns=columns_to_remove, inplace=True)

In [1478]:
data11 = add_sum(LAS11_subset, SMPS11_subset)

In [1479]:
#binning data11 into groups
nbins = 100
bins_1000 = np.linspace(data11.GPS_Altitude_THORNHILL.min(), data11.GPS_Altitude_THORNHILL.max(), nbins+1)
data11['Bins'] = pd.cut(data11['GPS_Altitude_THORNHILL'], bins_1000)
data11_1000 = data11.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data11_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data11_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100011 = data11_1000[start_index:stop_index]
DAY1000_subset_100011


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(137.295, 183.191]",2022-06-11 15:24:38.000,2022-06-11 15:24:39.000,139.4050,361.070,266.730,293.280,356.680,440.430,536.660,597.040,...,0.0,414.78,355.25,630.23,578.32,645.64,573.08,628.04,487.98,318.83
"(183.191, 229.087]",2022-06-11 15:25:26.500,2022-06-11 15:25:27.500,206.6225,204.210,232.510,270.975,313.710,410.970,537.020,476.360,...,0.0,414.78,355.25,630.23,578.32,645.64,573.08,628.04,487.98,318.83
"(229.087, 274.983]",2022-06-11 15:25:52.000,2022-06-11 15:25:53.000,248.9200,240.180,232.650,271.650,270.020,419.670,459.810,517.100,...,168.3,185.47,566.79,382.75,286.57,572.76,377.52,336.59,55.13,216.34
"(274.983, 320.88]",2022-06-11 15:26:15.000,2022-06-11 15:26:16.000,294.7700,240.930,198.490,188.540,298.900,362.510,408.690,515.860,...,168.3,185.47,566.79,382.75,286.57,572.76,377.52,336.59,55.13,216.34
"(320.88, 366.776]",2022-06-11 15:26:29.000,2022-06-11 15:26:30.000,343.1150,215.350,199.020,228.940,269.550,419.790,462.790,489.350,...,0.0,125.39,148.91,264.96,395.25,567.05,259.64,445.15,170.63,414.56
"(366.776, 412.672]",2022-06-11 15:26:42.500,2022-06-11 15:26:43.500,394.6075,239.550,199.770,291.915,328.975,392.580,507.730,488.555,...,0.0,125.39,148.91,264.96,395.25,567.05,259.64,445.15,170.63,414.56
"(412.672, 458.568]",2022-06-11 15:26:58.000,2022-06-11 15:26:59.000,434.9900,264.840,297.870,250.160,298.460,402.080,426.660,432.900,...,0.0,125.39,148.91,264.96,395.25,567.05,259.64,445.15,170.63,414.56
"(458.568, 504.464]",2022-06-11 15:27:15.500,2022-06-11 15:27:16.500,479.7575,324.825,299.205,281.555,329.165,363.185,445.360,407.595,...,0.0,125.39,148.91,264.96,395.25,567.05,259.64,445.15,170.63,414.56
"(504.464, 550.36]",2022-06-11 15:27:29.500,2022-06-11 15:27:30.500,527.8900,335.610,298.540,333.890,359.765,373.385,433.730,393.810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1480]:
DAY1000_subset_100011 = DAY1000_subset_100011.reset_index(drop=True)

In [1481]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data11_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data11_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300011 = data11_1000[start_index:stop_index]
DAY3000_subset_300011

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(1009.322, 1055.218]",2022-06-11 15:29:30.500,2022-06-11 15:29:31.500,1037.675,360.680,365.395,323.800,343.640,400.820,333.400,271.635,...,0.00,143.18,81.83,221.19,560.49,142.21,299.48,213.38,304.18,294.95
"(1055.218, 1101.114]",2022-06-11 15:29:43.500,2022-06-11 15:29:44.500,1071.975,324.335,315.945,396.080,358.835,295.855,313.840,298.315,...,0.00,143.18,81.83,221.19,560.49,142.21,299.48,213.38,304.18,294.95
"(1101.114, 1147.01]",2022-06-11 15:29:57.500,2022-06-11 15:29:58.500,1128.065,383.790,315.095,343.835,328.860,353.405,286.895,298.675,...,0.00,143.18,81.83,221.19,560.49,142.21,299.48,213.38,304.18,294.95
"(1147.01, 1192.906]",2022-06-11 15:30:12.500,2022-06-11 15:30:13.500,1171.605,347.480,365.705,291.885,314.220,286.480,259.005,298.440,...,0.00,143.18,81.83,221.19,560.49,142.21,299.48,213.38,304.18,294.95
"(1192.906, 1238.803]",2022-06-11 15:30:26.000,2022-06-11 15:30:27.000,1213.630,383.570,332.390,291.260,357.230,286.950,352.120,325.130,...,0.00,143.18,81.83,221.19,560.49,142.21,299.48,213.38,304.18,294.95
"(1238.803, 1284.699]",2022-06-11 15:30:37.500,2022-06-11 15:30:38.500,1261.425,348.460,299.225,271.575,268.765,324.975,277.755,257.890,...,6.57,243.10,268.52,324.31,261.74,735.40,449.74,378.16,663.50,527.70
"(1284.699, 1330.595]",2022-06-11 15:30:48.000,2022-06-11 15:30:49.000,1310.200,384.990,365.690,374.850,388.370,344.750,315.450,326.470,...,6.57,243.10,268.52,324.31,261.74,735.40,449.74,378.16,663.50,527.70
"(1330.595, 1376.491]",2022-06-11 15:31:00.000,2022-06-11 15:31:01.000,1354.910,408.970,432.020,418.480,419.410,401.570,388.380,298.960,...,6.57,243.10,268.52,324.31,261.74,735.40,449.74,378.16,663.50,527.70
"(1376.491, 1422.387]",2022-06-11 15:31:12.500,2022-06-11 15:31:13.500,1396.245,480.000,465.135,448.475,358.150,440.575,397.445,366.085,...,6.57,243.10,268.52,324.31,261.74,735.40,449.74,378.16,663.50,527.70


In [1482]:
DAY3000_subset_300011 = DAY3000_subset_300011.reset_index(drop=True)

In [1483]:
LAS11b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220611_RA_L2_20230131T122329.ict')
start_index = np.where(LAS11b["Time_Start"]==pd.to_datetime("2022-06-11 20:08:00"))
stop_index = np.where(LAS11b["Time_Start"]==pd.to_datetime("2022-06-11 20:15:39"))
    
LAS11b_subset = LAS11b[start_index[0][0]:stop_index[0][0]]
LAS11b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
9830,2022-06-11 20:08:00,2022-06-11 20:08:01,127.335,221.80,76.42,21.57,360.67,299.46,229.29,508.51,...,25.95,0.00,34.65,0.00,0.00,10.39,0.0,0.0,10.39,1.098
9831,2022-06-11 20:08:01,2022-06-11 20:08:02,125.935,235.86,40.84,4.33,529.11,299.29,563.29,448.70,...,0.00,17.33,17.33,25.98,0.00,10.39,0.0,0.0,0.00,1.098
9832,2022-06-11 20:08:02,2022-06-11 20:08:03,124.040,204.25,44.60,5.03,288.74,398.56,229.24,568.43,...,25.96,34.65,34.66,25.99,0.00,10.40,0.0,0.0,0.00,1.098
9833,2022-06-11 20:08:03,2022-06-11 20:08:04,121.770,196.66,41.31,5.07,409.53,366.03,229.81,389.64,...,25.99,17.34,34.69,0.00,10.41,10.41,0.0,0.0,0.00,1.098
9834,2022-06-11 20:08:04,2022-06-11 20:08:05,119.225,216.98,65.36,19.35,457.56,532.68,292.73,269.57,...,25.98,0.00,52.07,0.00,0.00,0.00,0.0,0.0,10.41,1.098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10284,2022-06-11 20:15:34,2022-06-11 20:15:35,2831.710,90.34,6.70,0.25,551.81,431.03,229.07,149.15,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.430
10285,2022-06-11 20:15:35,2022-06-11 20:15:36,2823.550,84.79,6.30,0.20,408.76,398.68,250.70,89.80,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.429
10286,2022-06-11 20:15:36,2022-06-11 20:15:37,2815.510,86.65,7.64,0.48,551.85,331.36,292.15,149.18,...,0.00,0.00,17.29,0.00,0.00,0.00,0.0,0.0,0.00,1.428
10287,2022-06-11 20:15:37,2022-06-11 20:15:38,2808.060,87.04,6.79,0.23,264.77,498.54,251.58,149.66,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,1.426


In [1484]:
SMPS11b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220611_RA_L2_20230131T122822.ict')
start_index = np.where(SMPS11b["Time_Start"]==pd.to_datetime("2022-06-11 20:08:00"))
stop_index = np.where(SMPS11b["Time_Start"]==pd.to_datetime("2022-06-11 20:15:39"))
    
SMPS11b_subset = SMPS11b[start_index[0][0]:stop_index[0][0]]
SMPS11b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
9830,2022-06-11 20:08:00,2022-06-11 20:08:01,127.335,168.50,1.73,0.02,0.0,0.0,0.0,0.00,...,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95,1.111
9831,2022-06-11 20:08:01,2022-06-11 20:08:02,125.935,168.50,1.73,0.02,0.0,0.0,0.0,0.00,...,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95,1.111
9832,2022-06-11 20:08:02,2022-06-11 20:08:03,124.040,168.50,1.73,0.02,0.0,0.0,0.0,0.00,...,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95,1.111
9833,2022-06-11 20:08:03,2022-06-11 20:08:04,121.770,168.50,1.73,0.02,0.0,0.0,0.0,0.00,...,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95,1.111
9834,2022-06-11 20:08:04,2022-06-11 20:08:05,119.225,168.50,1.73,0.02,0.0,0.0,0.0,0.00,...,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95,1.111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10284,2022-06-11 20:15:34,2022-06-11 20:15:35,2831.710,630.45,2.29,0.02,NaN,NaN,0.0,3212.89,...,482.55,564.96,476.38,488.14,278.26,349.31,222.79,343.72,352.54,1.442
10285,2022-06-11 20:15:35,2022-06-11 20:15:36,2823.550,630.45,2.29,0.02,NaN,NaN,0.0,3212.89,...,482.55,564.96,476.38,488.14,278.26,349.31,222.79,343.72,352.54,1.442
10286,2022-06-11 20:15:36,2022-06-11 20:15:37,2815.510,630.45,2.29,0.02,NaN,NaN,0.0,3212.89,...,482.55,564.96,476.38,488.14,278.26,349.31,222.79,343.72,352.54,1.442
10287,2022-06-11 20:15:37,2022-06-11 20:15:38,2808.060,630.45,2.29,0.02,NaN,NaN,0.0,3212.89,...,482.55,564.96,476.38,488.14,278.26,349.31,222.79,343.72,352.54,1.442


In [1485]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS11b_subset.drop(columns=columns_to_remove, inplace=True)

In [1486]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS11b_subset.drop(columns=columns_to_remove, inplace=True)

In [1487]:
data11b = add_sum(LAS11b_subset, SMPS11b_subset)

In [1488]:
#binning data11b into groups
nbins = 100
bins_1000 = np.linspace(data11b.GPS_Altitude_THORNHILL.min(), data11b.GPS_Altitude_THORNHILL.max(), nbins+1)
data11b['Bins'] = pd.cut(data11b['GPS_Altitude_THORNHILL'], bins_1000)
data11b_1000 = data11b.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data11b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data11b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100011b = data11b_1000[start_index:stop_index]
DAY1000_subset_100011b


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(108.925, 139.393]",2022-06-11 20:08:07.500,2022-06-11 20:08:08.500,117.8975,300.355,299.345,270.715,343.930,342.900,352.550,407.250,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(139.393, 169.862]",2022-06-11 20:08:18.500,2022-06-11 20:08:19.500,155.3050,215.815,281.230,207.695,282.920,266.745,295.745,271.110,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(169.862, 200.33]",2022-06-11 20:08:22.000,2022-06-11 20:08:23.000,184.5700,216.400,132.340,208.510,180.010,210.270,370.530,407.440,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(200.33, 230.799]",2022-06-11 20:08:25.500,2022-06-11 20:08:26.500,214.7325,155.820,182.520,198.155,209.445,286.155,286.475,285.195,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(230.799, 261.267]",2022-06-11 20:08:29.000,2022-06-11 20:08:30.000,247.0700,192.820,232.920,292.220,269.470,267.310,332.930,325.340,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(261.267, 291.736]",2022-06-11 20:08:32.000,2022-06-11 20:08:33.000,275.3900,215.940,132.890,188.190,238.860,402.210,444.070,408.230,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(291.736, 322.204]",2022-06-11 20:08:36.000,2022-06-11 20:08:37.000,308.0950,191.910,265.520,187.540,269.320,438.700,371.170,352.350,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(322.204, 352.673]",2022-06-11 20:08:41.000,2022-06-11 20:08:42.000,332.6500,168.850,166.580,250.210,329.290,304.670,407.810,351.680,...,189.16,82.86,476.70,253.02,368.03,686.03,429.41,370.77,164.00,212.95
"(352.673, 383.141]",2022-06-11 20:08:45.000,2022-06-11 20:08:46.000,364.2550,192.470,299.170,313.490,239.420,363.740,499.680,434.090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1489]:
DAY1000_subset_100011b = DAY1000_subset_100011b.reset_index(drop=True)

In [1490]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data11b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data11b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300011b = data11b_1000[start_index:stop_index]
DAY3000_subset_300011b

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(992.51, 1022.978]",2022-06-11 20:10:20.500,2022-06-11 20:10:21.500,1007.105,589.760,597.485,636.575,403.775,496.600,463.320,434.715,...,188.61,259.12,738.760,594.34,642.39,472.360,926.22,517.360,467.030,701.21
"(1022.978, 1053.447]",2022-06-11 20:10:24.000,2022-06-11 20:10:25.000,1034.660,625.900,532.200,520.850,478.530,440.050,407.420,488.610,...,188.61,259.12,738.760,594.34,642.39,472.360,926.22,517.360,467.030,701.21
"(1053.447, 1083.915]",2022-06-11 20:10:27.500,2022-06-11 20:10:28.500,1068.345,432.845,632.290,511.115,448.370,468.175,426.615,366.825,...,188.61,259.12,738.760,594.34,642.39,472.360,926.22,517.360,467.030,701.21
"(1083.915, 1114.384]",2022-06-11 20:10:31.000,2022-06-11 20:10:32.000,1100.200,455.690,432.570,395.760,418.250,420.620,295.200,298.900,...,188.61,259.12,738.760,594.34,642.39,472.360,926.22,517.360,467.030,701.21
"(1114.384, 1144.852]",2022-06-11 20:10:34.500,2022-06-11 20:10:35.500,1130.580,528.660,497.430,417.565,417.660,334.240,369.905,407.265,...,188.61,259.12,738.760,594.34,642.39,472.360,926.22,517.360,467.030,701.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2972.959, 3003.428]",2022-06-11 20:14:53.500,2022-06-11 20:14:54.500,2989.120,1176.720,830.315,500.795,417.700,200.480,185.230,67.840,...,336.68,498.68,581.695,608.53,983.20,479.065,646.18,478.705,409.595,517.17
"(3003.428, 3033.896]",2022-06-11 20:15:11.500,2022-06-11 20:15:12.500,3019.390,1344.930,978.545,646.790,492.695,239.360,120.380,95.050,...,309.81,482.55,564.960,476.38,488.14,278.260,349.31,222.790,343.720,352.54
"(3033.896, 3064.365]",2022-06-11 20:15:06.500,2022-06-11 20:15:07.500,3050.175,1643.970,1077.785,699.075,477.425,305.140,157.205,108.440,...,309.81,482.55,564.960,476.38,488.14,278.260,349.31,222.790,343.720,352.54


In [1491]:
DAY3000_subset_300011b = DAY3000_subset_300011b.reset_index(drop=True)

In [1492]:
LAS13 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220613_RA_L1_20230131T122330.ict')
start_index = np.where(LAS13["Time_Start"]==pd.to_datetime("2022-06-13 14:30:00"))
stop_index = np.where(LAS13["Time_Start"]==pd.to_datetime("2022-06-13 14:46:00"))
    
LAS13_subset = LAS13[start_index[0][0]:stop_index[0][0]]
LAS13_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10015,2022-06-13 14:30:00,2022-06-13 14:30:01,141.795,131.71,27.62,3.41,192.21,166.12,313.01,209.39,...,0.00,0.0,17.30,12.97,0.0,10.38,0.0,0.0,0.0,1.095
10016,2022-06-13 14:30:01,2022-06-13 14:30:02,141.725,102.29,17.02,1.30,167.96,165.89,166.53,178.91,...,25.88,0.0,17.27,12.96,0.0,0.00,0.0,0.0,0.0,1.095
10017,2022-06-13 14:30:02,2022-06-13 14:30:03,141.615,105.52,15.81,0.81,216.74,66.40,104.18,89.66,...,25.98,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.095
10018,2022-06-13 14:30:03,2022-06-13 14:30:04,141.460,112.04,12.16,0.43,72.10,299.21,292.38,209.03,...,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.095
10019,2022-06-13 14:30:04,2022-06-13 14:30:05,141.275,86.97,10.01,0.36,120.48,166.52,84.06,89.89,...,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10970,2022-06-13 14:45:55,2022-06-13 14:45:56,4504.770,430.34,21.28,0.51,3362.18,1957.49,1355.04,776.95,...,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.693
10971,2022-06-13 14:45:56,2022-06-13 14:45:57,4514.670,462.29,21.98,0.48,3271.92,2428.19,1483.80,927.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.695
10972,2022-06-13 14:45:57,2022-06-13 14:45:58,4523.980,442.97,20.38,0.45,3437.66,2492.47,1379.01,717.22,...,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.696
10973,2022-06-13 14:45:58,2022-06-13 14:45:59,4532.940,398.49,20.82,0.75,3126.47,2159.63,1254.09,598.78,...,0.00,0.0,17.33,0.00,0.0,0.00,0.0,0.0,0.0,1.698


In [1493]:
SMPS13 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220613_RA_L1_20230131T122823.ict')
start_index = np.where(SMPS13["Time_Start"]==pd.to_datetime("2022-06-13 14:30:00"))
stop_index = np.where(SMPS13["Time_Start"]==pd.to_datetime("2022-06-13 14:46:00"))
    
SMPS13_subset = SMPS13[start_index[0][0]:stop_index[0][0]]
SMPS13_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10015,2022-06-13 14:30:00,2022-06-13 14:30:01,141.795,147.5,1.37,0.01,0.0,0.0,0.0,0.0,...,163.18,417.79,233.83,314.72,423.38,216.21,141.55,217.55,236.81,1.095
10016,2022-06-13 14:30:01,2022-06-13 14:30:02,141.725,147.5,1.37,0.01,0.0,0.0,0.0,0.0,...,163.18,417.79,233.83,314.72,423.38,216.21,141.55,217.55,236.81,1.095
10017,2022-06-13 14:30:02,2022-06-13 14:30:03,141.615,147.5,1.37,0.01,0.0,0.0,0.0,0.0,...,163.18,417.79,233.83,314.72,423.38,216.21,141.55,217.55,236.81,1.095
10018,2022-06-13 14:30:03,2022-06-13 14:30:04,141.460,147.5,1.37,0.01,0.0,0.0,0.0,0.0,...,163.18,417.79,233.83,314.72,423.38,216.21,141.55,217.55,236.81,1.095
10019,2022-06-13 14:30:04,2022-06-13 14:30:05,141.275,147.5,1.37,0.01,0.0,0.0,0.0,0.0,...,163.18,417.79,233.83,314.72,423.38,216.21,141.55,217.55,236.81,1.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10970,2022-06-13 14:45:55,2022-06-13 14:45:56,4504.770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10971,2022-06-13 14:45:56,2022-06-13 14:45:57,4514.670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10972,2022-06-13 14:45:57,2022-06-13 14:45:58,4523.980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10973,2022-06-13 14:45:58,2022-06-13 14:45:59,4532.940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.727


In [1494]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS13_subset.drop(columns=columns_to_remove, inplace=True)

In [1495]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS13_subset.drop(columns=columns_to_remove, inplace=True)

In [1496]:
data13 = add_sum(LAS13_subset, SMPS13_subset)

In [1497]:
#binning data13 into groups
nbins = 100
bins_1000 = np.linspace(data13.GPS_Altitude_THORNHILL.min(), data13.GPS_Altitude_THORNHILL.max(), nbins+1)
data13['Bins'] = pd.cut(data13['GPS_Altitude_THORNHILL'], bins_1000)
data13_1000 = data13.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data13_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data13_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100013 = data13_1000[start_index:stop_index]
DAY1000_subset_100013


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(138.195, 182.23]",2022-06-13 14:30:32.000,2022-06-13 14:30:33.000,138.9350,120.480,165.890,167.490,180.190,248.270,278.580,300.170,...,281.52,163.18,417.79,233.83,314.72,423.38,216.21,141.55,217.55,236.81
"(182.23, 226.264]",2022-06-13 14:31:11.500,2022-06-13 14:31:12.500,199.8125,119.945,133.080,208.065,179.990,268.395,322.940,351.635,...,186.02,78.08,329.97,279.85,410.64,491.27,225.56,303.22,226.70,104.82
"(226.264, 270.299]",2022-06-13 14:31:24.000,2022-06-13 14:31:25.000,251.1650,96.350,132.940,146.250,209.760,210.630,352.660,351.510,...,186.02,78.08,329.97,279.85,410.64,491.27,225.56,303.22,226.70,104.82
"(270.299, 314.334]",2022-06-13 14:31:36.500,2022-06-13 14:31:37.500,292.3100,108.000,116.190,145.990,179.505,209.705,277.810,380.255,...,186.02,78.08,329.97,279.85,410.64,491.27,225.56,303.22,226.70,104.82
"(314.334, 358.368]",2022-06-13 14:31:48.000,2022-06-13 14:31:49.000,338.4650,95.800,132.410,104.760,178.970,189.740,276.820,299.450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(358.368, 402.403]",2022-06-13 14:32:01.000,2022-06-13 14:32:02.000,380.4400,96.140,66.590,125.370,119.680,209.690,315.710,352.870,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63
"(402.403, 446.438]",2022-06-13 14:32:13.000,2022-06-13 14:32:14.000,422.4850,96.150,99.620,145.810,209.500,191.430,241.190,271.200,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63
"(446.438, 490.472]",2022-06-13 14:32:22.000,2022-06-13 14:32:23.000,469.4600,120.340,132.240,104.590,119.760,191.410,259.600,271.810,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63
"(490.472, 534.507]",2022-06-13 14:32:33.000,2022-06-13 14:32:34.000,513.9000,120.450,165.600,146.690,178.730,230.090,222.190,270.550,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63


In [1498]:
DAY1000_subset_100013 = DAY1000_subset_100013.reset_index(drop=True)

In [1499]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data13_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data13_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300013 = data13_1000[start_index:stop_index]
DAY3000_subset_300013

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(1018.888, 1062.923]",2022-06-13 14:34:55.500,2022-06-13 14:34:56.500,1043.195,335.325,298.530,249.400,238.625,267.500,296.260,244.735,...,0.00,162.88,72.37,119.76,386.95,317.93,385.14,555.48,331.36,509.38
"(1062.923, 1106.957]",2022-06-13 14:35:07.500,2022-06-13 14:35:08.500,1085.090,300.365,282.305,271.495,253.930,286.215,259.250,285.785,...,0.00,162.88,72.37,119.76,386.95,317.93,385.14,555.48,331.36,509.38
"(1106.957, 1150.992]",2022-06-13 14:35:20.000,2022-06-13 14:35:21.000,1131.920,384.200,299.400,353.840,238.680,285.800,258.740,325.930,...,0.00,162.88,72.37,119.76,386.95,317.93,385.14,555.48,331.36,509.38
"(1150.992, 1195.027]",2022-06-13 14:35:35.000,2022-06-13 14:35:36.000,1172.530,408.850,297.810,333.110,238.160,266.030,185.410,216.840,...,0.00,162.88,72.37,119.76,386.95,317.93,385.14,555.48,331.36,509.38
"(1195.027, 1239.061]",2022-06-13 14:35:49.500,2022-06-13 14:35:50.500,1216.515,336.565,265.315,354.740,268.755,333.410,240.970,298.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1239.061, 1283.096]",2022-06-13 14:36:04.000,2022-06-13 14:36:05.000,1262.420,457.200,496.740,416.540,328.080,305.620,277.300,271.570,...,216.63,226.18,510.23,127.98,184.61,261.43,524.19,414.04,427.10,265.25
"(1283.096, 1327.131]",2022-06-13 14:36:18.000,2022-06-13 14:36:19.000,1303.960,478.640,331.530,375.460,358.040,325.130,276.690,271.310,...,216.63,226.18,510.23,127.98,184.61,261.43,524.19,414.04,427.10,265.25
"(1327.131, 1371.165]",2022-06-13 14:36:32.000,2022-06-13 14:36:33.000,1353.230,552.580,432.410,438.630,387.820,362.090,313.940,271.650,...,216.63,226.18,510.23,127.98,184.61,261.43,524.19,414.04,427.10,265.25
"(1371.165, 1415.2]",2022-06-13 14:36:48.500,2022-06-13 14:36:49.500,1387.600,468.755,563.260,375.640,357.360,334.610,268.290,203.310,...,216.63,226.18,510.23,127.98,184.61,261.43,524.19,414.04,427.10,265.25


In [1500]:
DAY3000_subset_300013 = DAY3000_subset_300013.reset_index(drop=True)

In [1501]:
LAS13b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220613_RA_L2_20230131T122330.ict')
start_index = np.where(LAS13b["Time_Start"]==pd.to_datetime("2022-06-13 19:46:00"))
stop_index = np.where(LAS13b["Time_Start"]==pd.to_datetime("2022-06-13 20:09:00"))
    
LAS13b_subset = LAS13b[start_index[0][0]:stop_index[0][0]]
LAS13b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10610,2022-06-13 19:46:00,2022-06-13 19:46:01,106.13,99.79,21.60,2.66,72.01,132.38,104.11,59.86,...,0.0,0.00,0.00,0.00,0.0,10.37,0.0,0.0,0.0,1.094
10611,2022-06-13 19:46:01,2022-06-13 19:46:02,105.59,84.42,12.34,0.71,0.00,66.42,187.50,89.59,...,0.0,0.00,0.00,12.97,0.0,0.00,0.0,0.0,0.0,1.093
10612,2022-06-13 19:46:02,2022-06-13 19:46:03,106.45,86.78,19.58,2.60,48.15,166.06,83.70,119.88,...,0.0,0.00,0.00,0.00,0.0,10.40,0.0,0.0,0.0,1.093
10613,2022-06-13 19:46:03,2022-06-13 19:46:04,108.91,108.14,16.95,1.02,72.01,132.84,124.65,208.86,...,0.0,17.28,17.29,0.00,0.0,0.00,0.0,0.0,0.0,1.094
10614,2022-06-13 19:46:04,2022-06-13 19:46:05,112.60,101.35,16.84,1.10,96.13,66.65,104.08,179.32,...,0.0,0.00,34.66,0.00,0.0,0.00,0.0,0.0,0.0,1.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11985,2022-06-13 20:08:55,2022-06-13 20:08:56,4558.09,524.73,28.24,0.75,3695.96,2522.39,1792.71,895.65,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.707
11986,2022-06-13 20:08:56,2022-06-13 20:08:57,4565.53,467.00,28.23,0.90,3051.23,2422.96,1480.79,836.63,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.709
11987,2022-06-13 20:08:57,2022-06-13 20:08:58,4572.98,463.28,23.34,0.57,3120.60,2854.48,1313.34,955.37,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.710
11988,2022-06-13 20:08:58,2022-06-13 20:08:59,4580.36,498.80,28.78,0.91,3290.11,2323.48,1648.07,1284.22,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,1.711


In [1502]:
SMPS13b = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220613_RA_L2_20230131T122823.ict')
start_index = np.where(SMPS13b["Time_Start"]==pd.to_datetime("2022-06-13 19:46:00"))
stop_index = np.where(SMPS13b["Time_Start"]==pd.to_datetime("2022-06-13 20:09:00"))
    
SMPS13b_subset = SMPS13[start_index[0][0]:stop_index[0][0]]
SMPS13b_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10610,2022-06-13 14:39:55,2022-06-13 14:39:56,1949.280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10611,2022-06-13 14:39:56,2022-06-13 14:39:57,1951.720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10612,2022-06-13 14:39:57,2022-06-13 14:39:58,1954.250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10613,2022-06-13 14:39:58,2022-06-13 14:39:59,1956.860,253.19,2.57,0.03,NaN,NaN,0.0,0.0,...,291.61,417.26,670.53,467.05,492.25,393.43,698.82,338.6,452.12,1.325
10614,2022-06-13 14:39:59,2022-06-13 14:40:00,1959.560,253.19,2.57,0.03,NaN,NaN,0.0,0.0,...,291.61,417.26,670.53,467.05,492.25,393.43,698.82,338.6,452.12,1.325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11755,2022-06-13 14:59:00,2022-06-13 14:59:01,-15.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11756,2022-06-13 14:59:01,2022-06-13 14:59:02,-18.330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11757,2022-06-13 14:59:02,2022-06-13 14:59:03,-21.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11758,2022-06-13 14:59:03,2022-06-13 14:59:04,-23.220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1503]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS13b_subset.drop(columns=columns_to_remove, inplace=True)

In [1504]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS13b_subset.drop(columns=columns_to_remove, inplace=True)

In [1505]:
data13b = add_sum(LAS13b_subset, SMPS13b_subset)

In [1506]:
#binning data13b into groups
nbins = 100
bins_1000 = np.linspace(data13b.GPS_Altitude_THORNHILL.min(), data13b.GPS_Altitude_THORNHILL.max(), nbins+1)
data13b['Bins'] = pd.cut(data13b['GPS_Altitude_THORNHILL'], bins_1000)
data13b_1000 = data13b.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data13b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data13b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100013b = data13b_1000[start_index:stop_index]
DAY1000_subset_100013b


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(105.59, 150.409]",2022-06-13 19:46:21.500,2022-06-13 19:46:22.500,119.6275,96.125,99.340,103.950,149.455,171.560,239.755,271.605,...,120.81,291.61,417.26,670.53,467.05,492.25,393.43,698.82,338.6,452.12
"(150.409, 195.228]",2022-06-13 19:46:50.000,2022-06-13 19:46:51.000,172.9900,96.150,99.340,62.530,119.460,171.880,167.230,216.740,...,120.81,291.61,417.26,670.53,467.05,492.25,393.43,698.82,338.6,452.12
"(195.228, 240.047]",2022-06-13 19:47:03.500,2022-06-13 19:47:04.500,218.5850,47.825,99.655,114.910,119.615,123.915,194.260,244.290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(240.047, 284.866]",2022-06-13 19:47:16.500,2022-06-13 19:47:17.500,261.7975,60.130,66.415,83.530,89.265,162.085,185.805,230.710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(284.866, 329.686]",2022-06-13 19:47:33.000,2022-06-13 19:47:34.000,309.0100,71.860,66.440,41.970,89.470,114.480,167.010,189.620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(329.686, 374.505]",2022-06-13 19:47:48.000,2022-06-13 19:47:49.000,352.0100,48.130,33.190,83.410,59.890,95.800,166.980,190.140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(374.505, 419.324]",2022-06-13 19:48:00.500,2022-06-13 19:48:01.500,396.1500,60.095,66.445,93.830,59.820,95.775,129.590,163.295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(419.324, 464.143]",2022-06-13 19:48:11.500,2022-06-13 19:48:12.500,442.3500,36.120,83.165,62.500,89.825,104.970,120.325,176.405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(464.143, 508.962]",2022-06-13 19:48:21.500,2022-06-13 19:48:22.500,489.9675,71.840,66.425,62.590,104.630,95.265,138.980,202.435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1507]:
DAY1000_subset_100013b = DAY1000_subset_100013b.reset_index(drop=True)

In [1508]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data13b_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data13b_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300013b = data13b_1000[start_index:stop_index]
DAY3000_subset_300013b

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(1001.972, 1046.791]",2022-06-13 19:51:08.500,2022-06-13 19:51:09.500,1023.140,287.825,283.105,291.665,268.975,229.310,240.440,203.230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1046.791, 1091.61]",2022-06-13 19:51:18.500,2022-06-13 19:51:19.500,1069.170,552.890,548.020,459.905,476.960,400.650,305.440,312.130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1091.61, 1136.429]",2022-06-13 19:51:28.500,2022-06-13 19:51:29.500,1112.985,467.880,382.095,365.360,329.455,286.685,286.990,270.935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1136.429, 1181.248]",2022-06-13 19:51:38.500,2022-06-13 19:51:39.500,1165.700,395.905,348.045,281.750,269.145,267.205,286.985,230.835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1181.248, 1226.067]",2022-06-13 19:51:52.000,2022-06-13 19:51:53.000,1203.510,454.570,299.390,292.010,268.000,267.050,240.850,297.750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1226.067, 1270.887]",2022-06-13 19:52:08.500,2022-06-13 19:52:09.500,1243.390,431.340,348.730,354.320,298.355,267.405,323.290,230.805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1270.887, 1315.706]",2022-06-13 19:52:25.000,2022-06-13 19:52:26.000,1291.870,361.050,364.930,354.880,269.190,324.420,258.860,217.130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1315.706, 1360.525]",2022-06-13 19:52:41.000,2022-06-13 19:52:42.000,1341.630,432.340,397.590,334.200,269.000,267.150,259.480,243.860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1360.525, 1405.344]",2022-06-13 19:52:54.500,2022-06-13 19:52:55.500,1385.490,323.800,448.130,375.870,284.285,296.425,240.625,230.835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1509]:
DAY3000_subset_300013b = DAY3000_subset_300013b.reset_index(drop=True)

In [1510]:
LAS14 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220614_RA_20230131T122331.ict')
start_index = np.where(LAS14["Time_Start"]==pd.to_datetime("2022-06-14 16:05:00"))
stop_index = np.where(LAS14["Time_Start"]==pd.to_datetime("2022-06-14 16:25:00"))
    
LAS14_subset = LAS14[start_index[0][0]:stop_index[0][0]]
LAS14_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
9363,2022-06-14 16:05:00,2022-06-14 16:05:01,117.000,139.02,14.28,0.48,216.02,199.07,207.97,298.58,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,1.100
9364,2022-06-14 16:05:01,2022-06-14 16:05:02,117.085,153.80,36.35,7.02,312.69,465.90,187.64,179.41,...,0.00,0.00,0.00,0.00,0.00,10.40,10.39,0.0,0.0,1.099
9365,2022-06-14 16:05:02,2022-06-14 16:05:03,117.190,124.98,25.56,4.87,358.84,132.10,270.30,356.98,...,0.00,17.23,0.00,0.00,0.00,0.00,10.34,0.0,0.0,1.099
9366,2022-06-14 16:05:03,2022-06-14 16:05:04,117.295,126.94,14.25,0.66,217.18,233.56,377.06,269.92,...,26.07,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,1.100
9367,2022-06-14 16:05:04,2022-06-14 16:05:05,117.380,160.88,28.24,3.59,383.66,364.28,270.76,566.15,...,0.00,0.00,17.26,25.90,0.00,10.35,0.00,0.0,0.0,1.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10558,2022-06-14 16:24:55,2022-06-14 16:24:56,8454.590,274.18,25.81,2.05,1875.01,1130.21,647.61,567.56,...,0.00,34.62,0.00,0.00,10.38,0.00,0.00,0.0,0.0,2.567
10559,2022-06-14 16:24:56,2022-06-14 16:24:57,8461.000,264.91,30.69,4.85,1418.22,1029.51,982.24,567.82,...,0.00,0.00,0.00,0.00,0.00,0.00,10.39,0.0,0.0,2.569
10560,2022-06-14 16:24:57,2022-06-14 16:24:58,8467.420,262.50,20.60,1.03,1730.17,897.41,730.81,568.24,...,0.00,17.30,17.31,0.00,0.00,0.00,0.00,0.0,0.0,2.571
10561,2022-06-14 16:24:58,2022-06-14 16:24:59,8473.860,280.41,17.94,0.77,1970.11,1395.86,771.27,448.27,...,0.00,0.00,0.00,12.98,0.00,0.00,0.00,0.0,0.0,2.573


In [1511]:
SMPS14 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220614_RA_20230131T122824.ict')
start_index = np.where(SMPS14["Time_Start"]==pd.to_datetime("2022-06-14 16:05:00"))
stop_index = np.where(SMPS14["Time_Start"]==pd.to_datetime("2022-06-14 16:25:00"))
    
SMPS14_subset = SMPS14[start_index[0][0]:stop_index[0][0]]
SMPS14_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
9363,2022-06-14 16:05:00,2022-06-14 16:05:01,117.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9364,2022-06-14 16:05:01,2022-06-14 16:05:02,117.085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9365,2022-06-14 16:05:02,2022-06-14 16:05:03,117.190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9366,2022-06-14 16:05:03,2022-06-14 16:05:04,117.295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9367,2022-06-14 16:05:04,2022-06-14 16:05:05,117.380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10558,2022-06-14 16:24:55,2022-06-14 16:24:56,8454.590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.604
10559,2022-06-14 16:24:56,2022-06-14 16:24:57,8461.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.604
10560,2022-06-14 16:24:57,2022-06-14 16:24:58,8467.420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.604
10561,2022-06-14 16:24:58,2022-06-14 16:24:59,8473.860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.604


In [1512]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS14_subset.drop(columns=columns_to_remove, inplace=True)

In [1513]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS14_subset.drop(columns=columns_to_remove, inplace=True)

In [1514]:
data14 = add_sum(LAS14_subset, SMPS14_subset)

In [1515]:
#binning data14 into groups
nbins = 100
bins_1000 = np.linspace(data14.GPS_Altitude_THORNHILL.min(), data14.GPS_Altitude_THORNHILL.max(), nbins+1)
data14['Bins'] = pd.cut(data14['GPS_Altitude_THORNHILL'], bins_1000)
data14_1000 = data14.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data14_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data14_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100014 = data14_1000[start_index:stop_index]
DAY1000_subset_100014


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(117.0, 200.633]",2022-06-14 16:05:20.000,2022-06-14 16:05:21.000,118.0250,263.880,198.400,250.820,268.770,342.090,314.990,354.220,...,566.15,798.39,1113.30,2030.87,1668.31,1065.54,824.47,190.28,0.0,0.0
"(200.633, 284.267]",2022-06-14 16:05:44.500,2022-06-14 16:05:45.500,243.7250,455.185,332.855,270.315,253.710,276.815,305.645,299.195,...,566.15,798.39,1113.30,2030.87,1668.31,1065.54,824.47,190.28,0.0,0.0
"(284.267, 367.9]",2022-06-14 16:05:55.000,2022-06-14 16:05:56.000,326.2200,287.380,232.240,250.390,239.150,304.340,351.110,271.910,...,566.15,798.39,1113.30,2030.87,1668.31,1065.54,824.47,190.28,0.0,0.0
"(367.9, 451.533]",2022-06-14 16:06:05.500,2022-06-14 16:06:06.500,410.0450,227.290,199.510,261.110,239.355,296.600,332.175,271.380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(451.533, 535.166]",2022-06-14 16:06:16.000,2022-06-14 16:06:17.000,493.6150,240.520,165.850,209.040,239.020,324.190,240.070,271.720,...,0.00,268.69,297.03,438.74,423.97,322.14,72.41,151.24,0.0,0.0
"(535.166, 618.8]",2022-06-14 16:06:27.500,2022-06-14 16:06:28.500,577.6025,215.695,182.590,249.395,328.400,315.545,258.900,284.780,...,0.00,268.69,297.03,438.74,423.97,322.14,72.41,151.24,0.0,0.0
"(618.8, 702.433]",2022-06-14 16:06:38.500,2022-06-14 16:06:39.500,662.5250,180.495,199.620,219.075,239.475,277.330,277.250,217.780,...,0.00,268.69,297.03,438.74,423.97,322.14,72.41,151.24,0.0,0.0
"(702.433, 786.066]",2022-06-14 16:06:48.500,2022-06-14 16:06:49.500,744.1025,156.255,265.775,177.700,179.640,296.260,259.375,244.100,...,0.00,268.69,297.03,438.74,423.97,322.14,72.41,151.24,0.0,0.0
"(786.066, 869.7]",2022-06-14 16:06:59.500,2022-06-14 16:07:00.500,825.6350,168.400,266.140,208.460,224.400,200.730,240.490,244.180,...,0.00,268.69,297.03,438.74,423.97,322.14,72.41,151.24,0.0,0.0


In [1516]:
DAY1000_subset_100014 = DAY1000_subset_100014.reset_index(drop=True)

In [1517]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data14_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data14_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300014 = data14_1000[start_index:stop_index]
DAY3000_subset_300014

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(1036.966, 1120.6]",2022-06-14 16:07:29.500,2022-06-14 16:07:30.500,1076.810,204.320,215.030,239.635,253.380,200.610,221.590,216.680,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
"(1120.6, 1204.233]",2022-06-14 16:07:39.000,2022-06-14 16:07:40.000,1162.240,287.930,232.260,229.460,209.470,209.870,277.250,217.710,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
"(1204.233, 1287.866]",2022-06-14 16:07:48.500,2022-06-14 16:07:49.500,1247.740,240.135,232.415,229.095,238.880,219.160,213.435,204.060,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
"(1287.866, 1371.5]",2022-06-14 16:07:58.500,2022-06-14 16:07:59.500,1331.135,228.275,232.525,218.950,209.155,172.315,175.825,162.920,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
"(1371.5, 1455.133]",2022-06-14 16:08:08.000,2022-06-14 16:08:09.000,1413.230,264.450,199.130,229.080,209.430,153.010,203.600,135.810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1455.133, 1538.766]",2022-06-14 16:08:17.500,2022-06-14 16:08:18.500,1499.150,288.345,249.180,250.320,179.495,219.410,185.505,176.415,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0
"(1538.766, 1622.399]",2022-06-14 16:08:27.000,2022-06-14 16:08:28.000,1580.140,265.220,265.980,229.360,268.320,172.320,129.660,81.510,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0
"(1622.399, 1706.033]",2022-06-14 16:08:36.500,2022-06-14 16:08:37.500,1663.475,264.380,232.605,218.530,194.280,153.210,129.805,81.460,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0
"(1706.033, 1789.666]",2022-06-14 16:08:46.500,2022-06-14 16:08:47.500,1749.545,252.480,216.590,187.665,224.355,200.720,148.230,135.775,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0


In [1518]:
DAY3000_subset_300014 = DAY3000_subset_300014.reset_index(drop=True)

In [1519]:
LAS17 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220617_RA_20230131T122332.ict')
start_index = np.where(LAS17["Time_Start"]==pd.to_datetime("2022-06-17 16:20:00"))
stop_index = np.where(LAS17["Time_Start"]==pd.to_datetime("2022-06-17 16:47:00"))
    
LAS17_subset = LAS17[start_index[0][0]:stop_index[0][0]]
LAS17_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nLAS_AmmSO4_ZIEMBA,sLAS_AmmSO4_ZIEMBA,vLAS_AmmSO4_ZIEMBA,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,...,LAS_Bin18_ZIEMBA,LAS_Bin19_ZIEMBA,LAS_Bin20_ZIEMBA,LAS_Bin21_ZIEMBA,LAS_Bin22_ZIEMBA,LAS_Bin23_ZIEMBA,LAS_Bin24_ZIEMBA,LAS_Bin25_ZIEMBA,LAS_Bin26_ZIEMBA,stdPT_ZIEMBA
10469,2022-06-17 16:20:00,2022-06-17 16:20:01,134.100,127.99,27.63,3.99,144.30,165.82,313.31,568.60,...,0.00,0.00,17.32,0.00,10.39,10.39,0.0,0.00,0.0,1.094
10470,2022-06-17 16:20:01,2022-06-17 16:20:02,133.795,132.55,29.79,3.97,120.10,365.65,271.18,389.37,...,0.00,34.60,51.91,0.00,0.00,10.38,0.0,0.00,0.0,1.094
10471,2022-06-17 16:20:02,2022-06-17 16:20:03,133.485,129.04,65.39,20.29,143.70,231.46,250.07,446.92,...,25.84,0.00,17.24,0.00,0.00,10.35,0.0,20.69,0.0,1.093
10472,2022-06-17 16:20:03,2022-06-17 16:20:04,133.170,147.64,35.04,5.60,288.83,199.61,542.90,299.24,...,0.00,34.66,0.00,13.00,0.00,20.79,0.0,0.00,0.0,1.094
10473,2022-06-17 16:20:04,2022-06-17 16:20:05,132.825,126.64,25.41,3.40,144.00,298.69,229.52,447.51,...,0.00,34.56,0.00,12.96,0.00,10.37,0.0,0.00,0.0,1.094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12084,2022-06-17 16:46:55,2022-06-17 16:46:56,7639.050,408.60,19.68,0.48,3360.07,2287.82,1249.91,507.51,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,2.391
12085,2022-06-17 16:46:56,2022-06-17 16:46:57,7645.810,379.91,17.68,0.43,3266.14,2091.24,1022.21,567.43,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,2.393
12086,2022-06-17 16:46:57,2022-06-17 16:46:58,7652.610,362.35,17.50,0.43,3294.11,1629.35,1273.34,358.67,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,2.394
12087,2022-06-17 16:46:58,2022-06-17 16:46:59,7659.440,360.80,18.31,0.45,2692.58,1627.39,1419.96,568.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,2.395


In [1520]:
SMPS17 = simple_read('C:/Users/taiwoajayi/Documents/Aeronet/bermuda/activate-mrg1_hu25_20220617_RA_20230131T122824.ict')
start_index = np.where(SMPS17["Time_Start"]==pd.to_datetime("2022-06-17 16:20:00"))
stop_index = np.where(SMPS17["Time_Start"]==pd.to_datetime("2022-06-17 16:47:00"))
    
SMPS17_subset = SMPS17[start_index[0][0]:stop_index[0][0]]
SMPS17_subset

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,nSMPS_ZIEMBA,sSMPS_ZIEMBA,vSMPS_ZIEMBA,SMPS_Bin01_ZIEMBA,SMPS_Bin02_ZIEMBA,SMPS_Bin03_ZIEMBA,SMPS_Bin04_ZIEMBA,...,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA,stdPT_ZIEMBA
10469,2022-06-17 16:20:00,2022-06-17 16:20:01,134.100,365.51,1.52,0.01,0.0,0.0,0.0,0.0,...,722.21,702.7,953.61,398.07,157.16,348.58,147.86,0.00,0.00,1.094
10470,2022-06-17 16:20:01,2022-06-17 16:20:02,133.795,365.51,1.52,0.01,0.0,0.0,0.0,0.0,...,722.21,702.7,953.61,398.07,157.16,348.58,147.86,0.00,0.00,1.094
10471,2022-06-17 16:20:02,2022-06-17 16:20:03,133.485,365.51,1.52,0.01,0.0,0.0,0.0,0.0,...,722.21,702.7,953.61,398.07,157.16,348.58,147.86,0.00,0.00,1.094
10472,2022-06-17 16:20:03,2022-06-17 16:20:04,133.170,365.51,1.52,0.01,0.0,0.0,0.0,0.0,...,722.21,702.7,953.61,398.07,157.16,348.58,147.86,0.00,0.00,1.094
10473,2022-06-17 16:20:04,2022-06-17 16:20:05,132.825,365.51,1.52,0.01,0.0,0.0,0.0,0.0,...,722.21,702.7,953.61,398.07,157.16,348.58,147.86,0.00,0.00,1.094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12084,2022-06-17 16:46:55,2022-06-17 16:46:56,7639.050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12085,2022-06-17 16:46:56,2022-06-17 16:46:57,7645.810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12086,2022-06-17 16:46:57,2022-06-17 16:46:58,7652.610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12087,2022-06-17 16:46:58,2022-06-17 16:46:59,7659.440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1521]:
# Columns to remove
columns_to_remove = ['nLAS_AmmSO4_ZIEMBA', 'sLAS_AmmSO4_ZIEMBA', 'vLAS_AmmSO4_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
LAS17_subset.drop(columns=columns_to_remove, inplace=True)

In [1522]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL', 'nSMPS_ZIEMBA', 'sSMPS_ZIEMBA', 'vSMPS_ZIEMBA', 'stdPT_ZIEMBA']

# Remove the specified columns in-place
SMPS17_subset.drop(columns=columns_to_remove, inplace=True)

In [1523]:
data17 = add_sum(LAS22_subset, SMPS22_subset)

In [1524]:
#binning data17 into groups
nbins = 100
bins_1000 = np.linspace(data17.GPS_Altitude_THORNHILL.min(), data17.GPS_Altitude_THORNHILL.max(), nbins+1)
data17['Bins'] = pd.cut(data17['GPS_Altitude_THORNHILL'], bins_1000)
data17_1000 = data17.groupby('Bins').median()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(data17_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data17_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_100017 = data17_1000[start_index:stop_index]
DAY1000_subset_100017


,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(105.985, 143.17]",2022-06-02 12:59:00.500,2022-06-02 12:59:01.500,113.9000,251.310,266.525,322.825,357.485,534.465,628.300,732.270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(143.17, 180.354]",2022-06-02 12:59:09.500,2022-06-02 12:59:10.500,160.4275,227.945,215.805,291.435,268.795,429.540,573.585,585.340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(180.354, 217.539]",2022-06-02 12:59:14.500,2022-06-02 12:59:15.500,199.8250,167.700,299.070,292.085,329.020,448.975,630.025,705.365,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(217.539, 254.723]",2022-06-02 12:59:20.500,2022-06-02 12:59:21.500,234.4575,204.510,248.745,345.775,285.185,535.540,685.960,799.030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(254.723, 291.908]",2022-06-02 12:59:26.000,2022-06-02 12:59:27.000,276.6850,216.140,232.550,334.480,416.160,438.380,792.010,705.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(291.908, 329.092]",2022-06-02 12:59:32.000,2022-06-02 12:59:33.000,304.7050,168.360,265.780,290.330,269.340,459.060,592.570,621.470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(329.092, 366.277]",2022-06-02 12:59:37.000,2022-06-02 12:59:38.000,345.2250,216.640,199.710,271.880,358.980,497.670,593.400,870.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(366.277, 403.461]",2022-06-02 12:59:40.500,2022-06-02 12:59:41.500,383.8625,215.320,280.905,260.645,328.430,563.660,703.970,664.435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(403.461, 440.646]",2022-06-02 12:59:45.500,2022-06-02 12:59:46.500,420.9575,179.675,232.285,302.175,328.590,468.330,657.300,718.195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1525]:
DAY1000_subset_100017 = DAY1000_subset_100017.reset_index(drop=True)

In [1526]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(data17_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(data17_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_300017 = data17_1000[start_index:stop_index]
DAY3000_subset_300017

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
Bins,,,,,,,,,,,,,,,,,,,,,
"(998.414, 1035.599]",2022-06-02 13:00:58.500,2022-06-02 13:00:59.500,1020.465,203.815,198.785,322.490,238.615,362.955,498.885,474.940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1035.599, 1072.783]",2022-06-02 13:01:03.000,2022-06-02 13:01:04.000,1057.400,216.480,232.960,269.150,328.970,458.460,516.820,485.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1072.783, 1109.968]",2022-06-02 13:01:08.000,2022-06-02 13:01:09.000,1089.620,215.860,231.820,229.970,418.960,362.300,426.930,353.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1109.968, 1147.152]",2022-06-02 13:01:13.000,2022-06-02 13:01:14.000,1128.500,215.800,232.520,250.460,238.680,285.960,332.590,379.380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1147.152, 1184.337]",2022-06-02 13:01:17.500,2022-06-02 13:01:18.500,1167.055,180.270,249.175,250.975,254.255,296.760,371.115,366.875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3266.672, 3303.856]",2022-06-02 13:06:03.500,2022-06-02 13:06:04.500,3284.205,661.360,449.255,396.495,284.805,287.310,176.080,149.595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3303.856, 3341.041]",2022-06-02 13:06:09.000,2022-06-02 13:06:10.000,3322.510,721.210,565.350,438.450,329.110,267.800,148.310,108.650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3341.041, 3378.225]",2022-06-02 13:06:13.500,2022-06-02 13:06:14.500,3359.940,913.680,614.850,594.945,553.380,315.570,203.740,190.170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1527]:
DAY3000_subset_300017 = DAY3000_subset_300017.reset_index(drop=True)

In [1528]:

# Concatenate the dataframes vertically (along rows)
ocean = pd.concat([DAY1000_subset_1000a, DAY1000_subset_1000b, DAY1000_subset_10002b], axis=0)

# Reset the index of the merged dataframe
ocean.reset_index(drop=True, inplace=True)
ocean

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-02 12:59:00.500,2022-06-02 12:59:01.500,113.9000,251.310,266.525,322.825,357.485,534.465,628.300,732.270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-02 12:59:09.500,2022-06-02 12:59:10.500,160.4275,227.945,215.805,291.435,268.795,429.540,573.585,585.340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-02 12:59:14.500,2022-06-02 12:59:15.500,199.8250,167.700,299.070,292.085,329.020,448.975,630.025,705.365,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-02 12:59:20.500,2022-06-02 12:59:21.500,234.4575,204.510,248.745,345.775,285.185,535.540,685.960,799.030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-02 12:59:26.000,2022-06-02 12:59:27.000,276.6850,216.140,232.550,334.480,416.160,438.380,792.010,705.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2022-06-02 18:40:19.500,2022-06-02 18:40:20.500,796.8525,264.005,265.645,312.845,388.880,439.925,526.135,597.735,...,102.6,486.11,290.51,270.33,152.71,477.86,246.41,311.24,264.72,338.81
73,2022-06-02 18:40:31.000,2022-06-02 18:40:32.000,826.1250,264.640,265.190,312.080,358.860,478.160,463.910,516.510,...,102.6,486.11,290.51,270.33,152.71,477.86,246.41,311.24,264.72,338.81
74,2022-06-02 18:40:41.000,2022-06-02 18:40:42.000,866.0400,240.170,232.750,291.290,387.550,383.130,575.430,516.570,...,102.6,486.11,290.51,270.33,152.71,477.86,246.41,311.24,264.72,338.81
75,2022-06-02 18:40:50.500,2022-06-02 18:40:51.500,900.6175,192.125,233.400,323.160,313.625,449.655,434.125,502.765,...,102.6,486.11,290.51,270.33,152.71,477.86,246.41,311.24,264.72,338.81


In [1529]:
# Concatenate the dataframes vertically (along rows)
ocean2 = pd.concat([DAY3000_subset_3000a, DAY3000_subset_3000b, DAY3000_subset_30002b], axis=0)
# Reset the index of the merged dataframe
ocean2.reset_index(drop=True, inplace=True)
ocean2

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-02 13:00:58.500,2022-06-02 13:00:59.500,1020.465,203.815,198.785,322.490,238.615,362.955,498.885,474.940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-02 13:01:03.000,2022-06-02 13:01:04.000,1057.400,216.480,232.960,269.150,328.970,458.460,516.820,485.480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-02 13:01:08.000,2022-06-02 13:01:09.000,1089.620,215.860,231.820,229.970,418.960,362.300,426.930,353.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-02 13:01:13.000,2022-06-02 13:01:14.000,1128.500,215.800,232.520,250.460,238.680,285.960,332.590,379.380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-02 13:01:17.500,2022-06-02 13:01:18.500,1167.055,180.270,249.175,250.975,254.255,296.760,371.115,366.875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2022-06-02 18:51:48.500,2022-06-02 18:51:49.500,3293.650,1033.785,647.165,511.985,253.800,210.210,83.280,54.295,...,384.09,685.37,659.81,796.36,1685.83,5189.77,1100.07,513.69,732.7,648.43
198,2022-06-02 18:51:54.000,2022-06-02 18:51:55.000,3329.670,1273.420,697.470,585.300,239.350,191.420,92.630,81.560,...,384.09,685.37,659.81,796.36,1685.83,5189.77,1100.07,513.69,732.7,648.43
199,2022-06-02 18:51:59.000,2022-06-02 18:52:00.000,3366.390,1488.890,864.390,626.490,358.650,248.820,111.010,54.340,...,384.09,685.37,659.81,796.36,1685.83,5189.77,1100.07,513.69,732.7,648.43
200,2022-06-02 18:52:04.500,2022-06-02 18:52:05.500,3406.315,1574.785,1147.670,710.305,433.920,267.730,92.615,81.475,...,384.09,685.37,659.81,796.36,1685.83,5189.77,1100.07,513.69,732.7,648.43


In [1533]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL']

# Remove the specified columns in-place
ocean.drop(columns=columns_to_remove, inplace=True)
ocean2.drop(columns=columns_to_remove, inplace=True)

In [1534]:
# Calculate median values along columns
ocean_median = ocean.median()

# Convert the Series to a DataFrame
ocean_median_df = ocean_median.to_frame().reset_index()
ocean_median_df.columns = ['Column', 'Median']
ocean_median_df

,Column,Median
0,LAS_Bin01_ZIEMBA,216.5100
1,LAS_Bin02_ZIEMBA,265.1550
2,LAS_Bin03_ZIEMBA,312.4000
3,LAS_Bin04_ZIEMBA,358.6000
4,LAS_Bin05_ZIEMBA,458.1550
5,LAS_Bin06_ZIEMBA,594.3900
6,LAS_Bin07_ZIEMBA,676.8750
7,LAS_Bin08_ZIEMBA,536.1700
8,LAS_Bin09_ZIEMBA,318.5200
9,LAS_Bin10_ZIEMBA,191.2100


In [1535]:
# Calculate median values along columns
ocean2_median = ocean2.median()

# Convert the Series to a DataFrame
ocean2_median_df = ocean2_median.to_frame().reset_index()
ocean2_median_df.columns = ['Column', 'Median']
ocean2_median_df

,Column,Median
0,LAS_Bin01_ZIEMBA,432.0775
1,LAS_Bin02_ZIEMBA,366.1075
2,LAS_Bin03_ZIEMBA,355.2550
3,LAS_Bin04_ZIEMBA,328.6200
4,LAS_Bin05_ZIEMBA,344.0550
5,LAS_Bin06_ZIEMBA,315.3650
6,LAS_Bin07_ZIEMBA,285.3075
7,LAS_Bin08_ZIEMBA,218.6975
8,LAS_Bin09_ZIEMBA,145.0500
9,LAS_Bin10_ZIEMBA,87.6100


In [1540]:
ocean1 = ocean.quantile(.25)
ocean_1 = ocean1.to_frame().reset_index()
ocean_1.columns = ['Column', 'Median']
ocean_1
ocean22 = ocean.quantile(.75)
ocean_2 = ocean22.to_frame().reset_index()
ocean_2.columns = ['Column', 'Median']
ocean_2
ocean_transport_plot_25 = ocean_median_df['Median'] - ocean_1['Median']
ocean_transport_plot_25
ocean_transport_plot_75 = ocean_2['Median'] - ocean_median_df['Median']
ocean_transport_plot_75


ocean3 = ocean2.quantile(.25)
ocean_3 = ocean3.to_frame().reset_index()
ocean_3.columns = ['Column', 'Median']
ocean_3
ocean4 = ocean2.quantile(.75)
ocean_4 = ocean4.to_frame().reset_index()
ocean_4.columns = ['Column', 'Median']
ocean_4
ocean2_transport_plot_25 = ocean2_median_df['Median'] - ocean_3['Median']
ocean2_transport_plot_25
ocean2_transport_plot_75 = ocean_4['Median'] - ocean2_median_df['Median']
ocean2_transport_plot_75

0     361.41000
1     264.79000
2     195.77375
3      90.57875
4      76.59625
5      91.29000
6      95.61125
7      76.17750
8      39.68000
9      23.92125
10      0.16375
11     17.27125
12      8.63500
13      0.00000
14      0.00000
15      0.00000
16      0.00000
17      0.00000
18      0.00000
19      0.00000
20      0.00000
21      0.00000
22      0.00000
23      0.00000
24      0.00000
25      0.00000
26          NaN
27      0.00000
28      0.00000
29      0.00000
30      0.00000
31      0.00000
32      0.00000
33      0.00000
34      0.00000
35      0.00000
36      0.00000
37      0.00000
38    301.50000
39     87.84000
40    163.04000
41    170.72500
42    116.56500
43    159.50000
44    295.69000
45    118.54000
46     58.85000
47    209.81000
48    152.10000
49    158.88000
50    176.31000
51    450.66750
52    167.82000
53     93.66000
54    335.13000
55    150.21500
Name: Median, dtype: float64

In [1541]:
# Concatenate the dataframes vertically (along rows)
North_America = pd.concat([DAY1000_subset_10007, DAY1000_subset_10007b, DAY1000_subset_10008, DAY1000_subset_100017], axis=0)
# Reset the index of the merged dataframe
North_America.reset_index(drop=True, inplace=True)
North_America

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-07 14:32:05.500,2022-06-07 14:32:06.500,130.6300,2032.270,2075.820,1973.565,1659.225,1592.380,1063.325,694.175,...,536.42,1054.51,1371.55,1616.39,1793.30,1074.02,1993.01,1998.35,1970.72,1742.82
1,2022-06-07 14:32:14.000,2022-06-07 14:32:15.000,172.0250,2350.470,2165.020,2186.610,1768.040,1662.370,1203.370,758.770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-07 14:32:27.500,2022-06-07 14:32:28.500,188.3900,2292.240,2206.645,2116.335,1849.310,1577.410,1138.130,760.205,...,917.52,868.04,1333.83,1222.31,1812.84,716.57,1338.19,2233.26,1809.81,1030.33
3,2022-06-07 14:32:40.000,2022-06-07 14:32:41.000,219.6400,2044.860,2218.550,1968.420,1525.650,1510.960,1003.890,892.980,...,917.52,868.04,1333.83,1222.31,1812.84,716.57,1338.19,2233.26,1809.81,1030.33
4,2022-06-07 14:32:48.000,2022-06-07 14:32:49.000,250.9800,2356.010,2315.050,2241.750,1945.240,1655.410,1059.260,677.630,...,917.52,868.04,1333.83,1222.31,1812.84,716.57,1338.19,2233.26,1809.81,1030.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2022-06-02 13:00:31.500,2022-06-02 13:00:32.500,793.3525,203.765,249.375,248.790,356.840,390.445,478.985,474.515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,2022-06-02 13:00:36.500,2022-06-02 13:00:37.500,832.9125,300.155,182.315,219.355,343.975,372.510,508.905,541.350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,2022-06-02 13:00:40.500,2022-06-02 13:00:41.500,870.2225,240.085,199.225,260.645,357.965,353.565,481.750,474.435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,2022-06-02 13:00:44.500,2022-06-02 13:00:45.500,907.3525,228.215,282.515,229.700,254.425,391.370,435.485,584.540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1542]:
# Concatenate the dataframes vertically (along rows)
North_America2 = pd.concat([DAY3000_subset_30007, DAY3000_subset_30007b, DAY3000_subset_30008, DAY3000_subset_300017], axis=0)

# Reset the index of the merged dataframe
North_America2.reset_index(drop=True, inplace=True)
North_America2

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-07 14:38:12.000,2022-06-07 14:38:13.000,1030.120,1490.670,1455.590,1849.820,1634.340,1395.52,981.430,647.840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-07 14:38:21.000,2022-06-07 14:38:22.000,1055.940,1102.160,1265.070,1690.460,1607.140,1411.47,945.350,758.820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-07 14:38:32.000,2022-06-07 14:38:33.000,1084.140,962.470,1327.310,1561.550,1793.100,1532.14,907.590,598.810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-07 14:38:42.500,2022-06-07 14:38:43.500,1107.855,959.755,1229.990,1499.810,1685.870,1290.73,899.715,529.895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-07 14:38:52.000,2022-06-07 14:38:53.000,1136.620,914.990,1131.110,1498.620,1490.640,1489.20,885.460,570.760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,2022-06-02 13:06:03.500,2022-06-02 13:06:04.500,3284.205,661.360,449.255,396.495,284.805,287.31,176.080,149.595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,2022-06-02 13:06:09.000,2022-06-02 13:06:10.000,3322.510,721.210,565.350,438.450,329.110,267.80,148.310,108.650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,2022-06-02 13:06:13.500,2022-06-02 13:06:14.500,3359.940,913.680,614.850,594.945,553.380,315.57,203.740,190.170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,2022-06-02 13:06:18.000,2022-06-02 13:06:19.000,3397.350,1203.030,731.270,731.390,568.900,439.18,240.520,190.190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1543]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL']

# Remove the specified columns in-place
North_America.drop(columns=columns_to_remove, inplace=True)
North_America2.drop(columns=columns_to_remove, inplace=True)

In [1544]:
# Calculate median values along columns
North_America_median = North_America.median()

# Convert the Series to a DataFrame
North_America_median_df = North_America_median.to_frame().reset_index()
North_America_median_df.columns = ['Column', 'Median']
North_America_median_df


# Calculate median values along columns
North_America2_median = North_America2.median()

# Convert the Series to a DataFrame
North_America2_median_df = North_America2_median.to_frame().reset_index()
North_America2_median_df.columns = ['Column', 'Median']
North_America2_median_df

,Column,Median
0,LAS_Bin01_ZIEMBA,1032.500
1,LAS_Bin02_ZIEMBA,764.040
2,LAS_Bin03_ZIEMBA,564.030
3,LAS_Bin04_ZIEMBA,448.090
4,LAS_Bin05_ZIEMBA,362.710
5,LAS_Bin06_ZIEMBA,259.040
6,LAS_Bin07_ZIEMBA,190.160
7,LAS_Bin08_ZIEMBA,142.890
8,LAS_Bin09_ZIEMBA,79.380
9,LAS_Bin10_ZIEMBA,52.530


In [1545]:
North_America1 = North_America.quantile(.25)
North_America_1 = North_America1.to_frame().reset_index()
North_America_1.columns = ['Column', 'Median']
North_America_1
North_America22 = North_America.quantile(.75)
North_America_2 = North_America22.to_frame().reset_index()
North_America_2.columns = ['Column', 'Median']
North_America_2
North_America_transport_plot_25 = North_America_median_df['Median'] - North_America_1['Median']
North_America_transport_plot_25
North_America_transport_plot_75 = North_America_2['Median'] - North_America_median_df['Median']
North_America_transport_plot_75


North_America3 = North_America2.quantile(.25)
North_America_3 = North_America3.to_frame().reset_index()
North_America_3.columns = ['Column', 'Median']
North_America_3
North_America4 = North_America2.quantile(.75)
North_America_4 = North_America4.to_frame().reset_index()
North_America_4.columns = ['Column', 'Median']
North_America_4
North_America2_transport_plot_25 = North_America2_median_df['Median'] - North_America_3['Median']
North_America2_transport_plot_25
North_America2_transport_plot_75 = North_America_4['Median'] - North_America2_median_df['Median']
North_America2_transport_plot_75

0     378.3975
1     374.3425
2     277.0825
3     178.6100
4     139.3475
5      92.2625
6      81.4375
7      71.0150
8      52.6350
9      30.5400
10     26.0350
11      8.7150
12     17.2900
13      0.0000
14      0.0000
15      0.0000
16      0.0000
17      0.0000
18      0.0000
19      0.0000
20      0.0000
21      0.0000
22      0.0000
23      0.0000
24      0.0000
25      0.0000
26         NaN
27      0.0000
28      0.0000
29      0.0000
30      0.0000
31      0.0000
32      0.0000
33      0.0000
34      0.0000
35      0.0000
36      0.0000
37    214.7600
38    167.9700
39     87.7000
40    166.8425
41    103.0700
42      8.4400
43     63.8400
44     59.6500
45    210.1600
46    122.2600
47    228.4000
48    224.0000
49    206.1775
50    216.6900
51    189.5250
52    440.8250
53    406.1200
54    657.4600
55    361.8200
Name: Median, dtype: float64

In [1547]:
# Concatenate the dataframes vertically (along rows)
Caribbean = pd.concat([DAY1000_subset_100010, DAY1000_subset_10001110b, DAY1000_subset_100010b, DAY1000_subset_100011, DAY1000_subset_100011b], axis=0)

# Reset the index of the merged dataframe
Caribbean.reset_index(drop=True, inplace=True)
Caribbean

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-10 15:17:25.000,2022-06-10 15:17:26.000,123.7050,192.300,199.230,249.770,327.140,381.050,461.730,408.660,...,375.00,306.29,440.89,322.11,614.07,812.49,519.63,197.71,53.83,211.18
1,2022-06-10 15:17:52.500,2022-06-10 15:17:53.500,173.7850,144.325,282.870,240.000,254.660,391.300,361.415,406.960,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
2,2022-06-10 15:17:58.000,2022-06-10 15:17:59.000,209.2900,215.620,230.760,272.220,238.780,362.540,368.350,433.770,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
3,2022-06-10 15:18:03.000,2022-06-10 15:18:04.000,244.1150,192.080,199.300,208.530,239.400,305.130,407.690,379.780,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
4,2022-06-10 15:18:07.500,2022-06-10 15:18:08.500,277.9725,96.000,165.715,197.485,194.450,313.935,489.690,365.755,...,178.11,486.65,170.00,552.52,600.60,718.31,433.85,362.03,218.43,161.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2022-06-11 20:09:53.000,2022-06-11 20:09:54.000,824.3850,385.360,430.040,396.570,506.560,418.880,408.250,514.290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,2022-06-11 20:09:58.000,2022-06-11 20:09:59.000,852.6800,407.070,429.380,414.490,327.590,474.930,351.120,487.550,...,188.61,259.12,738.76,594.34,642.39,472.36,926.22,517.36,467.03,701.21
101,2022-06-11 20:10:02.000,2022-06-11 20:10:03.000,884.2350,527.750,463.560,416.010,389.030,458.950,463.310,379.400,...,188.61,259.12,738.76,594.34,642.39,472.36,926.22,517.36,467.03,701.21
102,2022-06-11 20:10:05.500,2022-06-11 20:10:06.500,914.1000,539.520,615.115,500.170,477.775,458.915,416.420,421.255,...,188.61,259.12,738.76,594.34,642.39,472.36,926.22,517.36,467.03,701.21


In [1548]:
# Concatenate the dataframes vertically (along rows)
Caribbean2 = pd.concat([DAY3000_subset_300010,  DAY3000_subset_30001110b, DAY3000_subset_300010b, DAY3000_subset_300011, DAY3000_subset_300011b, DAY3000_subset_300013, DAY3000_subset_300013b], axis=0)

# Reset the index of the merged dataframe
Caribbean2.reset_index(drop=True, inplace=True)
Caribbean2

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-10 15:19:33.500,2022-06-10 15:19:34.500,1002.010,588.130,647.275,740.280,611.740,496.675,629.415,515.535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-10 15:19:37.500,2022-06-10 15:19:38.500,1036.915,782.030,798.865,636.765,734.140,554.875,669.350,462.375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-10 15:19:41.500,2022-06-10 15:19:42.500,1072.325,660.620,714.505,679.205,732.705,610.305,500.380,353.310,...,956.23,963.66,1070.83,1064.71,718.07,913.82,659.86,552.77,1091.01,952.28
3,2022-06-10 15:19:45.000,2022-06-10 15:19:46.000,1103.920,745.140,730.280,751.620,686.920,669.280,666.720,407.330,...,956.23,963.66,1070.83,1064.71,718.07,913.82,659.86,552.77,1091.01,952.28
4,2022-06-10 15:19:48.500,2022-06-10 15:19:49.500,1135.630,636.365,713.835,769.480,776.345,781.470,601.035,530.530,...,956.23,963.66,1070.83,1064.71,718.07,913.82,659.86,552.77,1091.01,952.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,2022-06-13 20:06:14.000,2022-06-13 20:06:15.000,3261.760,2165.980,1328.890,1065.520,745.780,401.360,222.700,163.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,2022-06-13 20:06:18.500,2022-06-13 20:06:19.500,3307.625,2027.100,1610.300,1157.260,671.150,505.605,323.535,203.580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,2022-06-13 20:06:23.000,2022-06-13 20:06:24.000,3355.810,3026.630,1892.170,1315.650,627.810,533.420,296.140,190.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,2022-06-13 20:06:29.500,2022-06-13 20:06:30.500,3402.535,2772.865,1843.075,1170.135,851.815,553.805,231.390,176.495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1549]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL']

# Remove the specified columns in-place
Caribbean.drop(columns=columns_to_remove, inplace=True)
Caribbean2.drop(columns=columns_to_remove, inplace=True)

In [1550]:
# Calculate median values along columns
Caribbean_median = Caribbean.median()

# Convert the Series to a DataFrame
Caribbean_median_df = Caribbean_median.to_frame().reset_index()
Caribbean_median_df.columns = ['Column', 'Median']
Caribbean_median_df


# Calculate median values along columns
Caribbean2_median = Caribbean2.median()

# Convert the Series to a DataFrame
Caribbean2_median_df = Caribbean2_median.to_frame().reset_index()
Caribbean2_median_df.columns = ['Column', 'Median']
Caribbean2_median_df

,Column,Median
0,LAS_Bin01_ZIEMBA,517.1025
1,LAS_Bin02_ZIEMBA,465.0075
2,LAS_Bin03_ZIEMBA,417.1350
3,LAS_Bin04_ZIEMBA,357.9600
4,LAS_Bin05_ZIEMBA,296.1750
5,LAS_Bin06_ZIEMBA,240.4150
6,LAS_Bin07_ZIEMBA,203.2700
7,LAS_Bin08_ZIEMBA,160.4400
8,LAS_Bin09_ZIEMBA,105.5750
9,LAS_Bin10_ZIEMBA,52.5350


In [1551]:
Caribbean1 = Caribbean.quantile(.25)
Caribbean_1 = Caribbean1.to_frame().reset_index()
Caribbean_1.columns = ['Column', 'Median']
Caribbean_1
Caribbean22 = Caribbean.quantile(.75)
Caribbean_2 = Caribbean22.to_frame().reset_index()
Caribbean_2.columns = ['Column', 'Median']
Caribbean_2
Caribbean_transport_plot_25 = Caribbean_median_df['Median'] - Caribbean_1['Median']
Caribbean_transport_plot_25
Caribbean_transport_plot_75 = Caribbean_2['Median'] - Caribbean_median_df['Median']
Caribbean_transport_plot_75


Caribbean3 = Caribbean2.quantile(.25)
Caribbean_3 = Caribbean3.to_frame().reset_index()
Caribbean_3.columns = ['Column', 'Median']
Caribbean_3
Caribbean4 = Caribbean2.quantile(.75)
Caribbean_4 = Caribbean4.to_frame().reset_index()
Caribbean_4.columns = ['Column', 'Median']
Caribbean_4
Caribbean2_transport_plot_25 = Caribbean2_median_df['Median'] - Caribbean_3['Median']
Caribbean2_transport_plot_25
Caribbean2_transport_plot_75 = Caribbean_4['Median'] - Caribbean2_median_df['Median']
Caribbean2_transport_plot_75

0     156.61750
1     199.24625
2     167.78375
3     134.87375
4     152.56625
5     145.71375
6     123.09000
7     107.04625
8      79.12000
9      43.82250
10     26.10875
11     17.33000
12      8.68375
13      0.00000
14      8.64500
15      0.00000
16      0.00000
17      0.00000
18     17.28000
19     17.31500
20      0.00000
21      0.00000
22      0.00000
23      0.00000
24      0.00000
25      0.00000
26          NaN
27      0.00000
28      0.00000
29      0.00000
30      0.00000
31      0.00000
32      0.00000
33      0.00000
34      0.00000
35      0.00000
36      0.00000
37      0.00000
38      0.00000
39      0.00000
40    140.37000
41    203.26500
42    114.73500
43     86.88000
44    162.86000
45    174.27000
46    295.35000
47    315.28250
48    220.71000
49    140.31000
50    162.81000
51    184.66000
52    243.98000
53    102.00000
54    228.43000
55    181.82000
Name: Median, dtype: float64

In [1552]:
Africa = pd.concat([DAY1000_subset_100013, DAY1000_subset_100013b, DAY1000_subset_100014])
# Reset the index of the merged dataframe
Africa.reset_index(drop=True, inplace=True)
Africa

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-13 14:30:32.000,2022-06-13 14:30:33.000,138.9350,120.480,165.890,167.490,180.190,248.270,278.580,300.170,...,281.52,163.18,417.79,233.83,314.72,423.38,216.21,141.55,217.55,236.81
1,2022-06-13 14:31:11.500,2022-06-13 14:31:12.500,199.8125,119.945,133.080,208.065,179.990,268.395,322.940,351.635,...,186.02,78.08,329.97,279.85,410.64,491.27,225.56,303.22,226.70,104.82
2,2022-06-13 14:31:24.000,2022-06-13 14:31:25.000,251.1650,96.350,132.940,146.250,209.760,210.630,352.660,351.510,...,186.02,78.08,329.97,279.85,410.64,491.27,225.56,303.22,226.70,104.82
3,2022-06-13 14:31:36.500,2022-06-13 14:31:37.500,292.3100,108.000,116.190,145.990,179.505,209.705,277.810,380.255,...,186.02,78.08,329.97,279.85,410.64,491.27,225.56,303.22,226.70,104.82
4,2022-06-13 14:31:48.000,2022-06-13 14:31:49.000,338.4650,95.800,132.410,104.760,178.970,189.740,276.820,299.450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-06-13 14:32:01.000,2022-06-13 14:32:02.000,380.4400,96.140,66.590,125.370,119.680,209.690,315.710,352.870,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63
6,2022-06-13 14:32:13.000,2022-06-13 14:32:14.000,422.4850,96.150,99.620,145.810,209.500,191.430,241.190,271.200,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63
7,2022-06-13 14:32:22.000,2022-06-13 14:32:23.000,469.4600,120.340,132.240,104.590,119.760,191.410,259.600,271.810,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63
8,2022-06-13 14:32:33.000,2022-06-13 14:32:34.000,513.9000,120.450,165.600,146.690,178.730,230.090,222.190,270.550,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63
9,2022-06-13 14:32:45.500,2022-06-13 14:32:46.500,556.2900,143.990,149.620,135.620,149.140,191.710,204.125,216.565,...,196.09,211.18,176.98,320.14,298.97,439.41,227.44,281.60,175.51,137.63


In [1555]:
Africa2 = DAY3000_subset_300014
Africa2

,Time_Start,Time_Stop,GPS_Altitude_THORNHILL,LAS_Bin01_ZIEMBA,LAS_Bin02_ZIEMBA,LAS_Bin03_ZIEMBA,LAS_Bin04_ZIEMBA,LAS_Bin05_ZIEMBA,LAS_Bin06_ZIEMBA,LAS_Bin07_ZIEMBA,...,SMPS_Bin21_ZIEMBA,SMPS_Bin22_ZIEMBA,SMPS_Bin23_ZIEMBA,SMPS_Bin24_ZIEMBA,SMPS_Bin25_ZIEMBA,SMPS_Bin26_ZIEMBA,SMPS_Bin27_ZIEMBA,SMPS_Bin28_ZIEMBA,SMPS_Bin29_ZIEMBA,SMPS_Bin30_ZIEMBA
0,2022-06-14 16:07:29.500,2022-06-14 16:07:30.500,1076.810,204.320,215.030,239.635,253.380,200.610,221.590,216.680,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
1,2022-06-14 16:07:39.000,2022-06-14 16:07:40.000,1162.240,287.930,232.260,229.460,209.470,209.870,277.250,217.710,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
2,2022-06-14 16:07:48.500,2022-06-14 16:07:49.500,1247.740,240.135,232.415,229.095,238.880,219.160,213.435,204.060,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
3,2022-06-14 16:07:58.500,2022-06-14 16:07:59.500,1331.135,228.275,232.525,218.950,209.155,172.315,175.825,162.920,...,107.46,290.20,211.88,76.61,281.59,359.16,138.88,289.30,0.00,0.0
4,2022-06-14 16:08:08.000,2022-06-14 16:08:09.000,1413.230,264.450,199.130,229.080,209.430,153.010,203.600,135.810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-06-14 16:08:17.500,2022-06-14 16:08:18.500,1499.150,288.345,249.180,250.320,179.495,219.410,185.505,176.415,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0
6,2022-06-14 16:08:27.000,2022-06-14 16:08:28.000,1580.140,265.220,265.980,229.360,268.320,172.320,129.660,81.510,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0
7,2022-06-14 16:08:36.500,2022-06-14 16:08:37.500,1663.475,264.380,232.605,218.530,194.280,153.210,129.805,81.460,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0
8,2022-06-14 16:08:46.500,2022-06-14 16:08:47.500,1749.545,252.480,216.590,187.665,224.355,200.720,148.230,135.775,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0
9,2022-06-14 16:08:56.000,2022-06-14 16:08:57.000,1831.050,264.870,199.670,208.480,149.630,190.850,148.180,81.400,...,545.04,399.68,127.11,412.68,320.59,414.18,251.64,132.92,0.00,0.0


In [1556]:
# Columns to remove
columns_to_remove = ['Time_Start', 'Time_Stop', 'GPS_Altitude_THORNHILL']

# Remove the specified columns in-place
Africa.drop(columns=columns_to_remove, inplace=True)
Africa2.drop(columns=columns_to_remove, inplace=True)

In [1557]:
# Calculate median values along columns
Africa_median = Africa.median()

# Convert the Series to a DataFrame
Africa_median_df = Africa_median.to_frame().reset_index()
Africa_median_df.columns = ['Column', 'Median']
Africa_median_df


# Calculate median values along columns
Africa2_median = Africa2.median()

# Convert the Series to a DataFrame
Africa2_median_df = Africa2_median.to_frame().reset_index()
Africa2_median_df.columns = ['Column', 'Median']
Africa2_median_df

,Column,Median
0,LAS_Bin01_ZIEMBA,210.1850
1,LAS_Bin02_ZIEMBA,166.3950
2,LAS_Bin03_ZIEMBA,135.6500
3,LAS_Bin04_ZIEMBA,119.4050
4,LAS_Bin05_ZIEMBA,86.0500
5,LAS_Bin06_ZIEMBA,55.4400
6,LAS_Bin07_ZIEMBA,40.7600
7,LAS_Bin08_ZIEMBA,35.6825
8,LAS_Bin09_ZIEMBA,26.4175
9,LAS_Bin10_ZIEMBA,17.4950


In [1558]:
Africa1 = Africa.quantile(.25)
Africa_1 = Africa1.to_frame().reset_index()
Africa_1.columns = ['Column', 'Median']
Africa_1
Africa22 = Africa.quantile(.75)
Africa_2 = Africa22.to_frame().reset_index()
Africa_2.columns = ['Column', 'Median']
Africa_2
Africa_transport_plot_25 = Africa_median_df['Median'] - Africa_1['Median']
Africa_transport_plot_25
Africa_transport_plot_75 = Africa_2['Median'] - Africa_median_df['Median']
Africa_transport_plot_75


Africa3 = Africa2.quantile(.25)
Africa_3 = Africa3.to_frame().reset_index()
Africa_3.columns = ['Column', 'Median']
Africa_3
Africa4 = Africa2.quantile(.75)
Africa_4 = Africa4.to_frame().reset_index()
Africa_4.columns = ['Column', 'Median']
Africa_4
Africa2_transport_plot_25 = Africa2_median_df['Median'] - Africa_3['Median']
Africa2_transport_plot_25
Africa2_transport_plot_75 = Africa_4['Median'] - Africa2_median_df['Median']
Africa2_transport_plot_75

0      45.18000
1      49.02500
2      82.98500
3      78.59375
4      86.26625
5      92.75250
6      74.63000
7      71.52625
8      26.50875
9       0.04125
10      0.00000
11      0.00000
12      0.00000
13      0.00000
14      0.00000
15      3.24750
16      0.00000
17      0.00000
18     17.30125
19     28.11875
20      8.10625
21      0.00000
22      0.00000
23      0.00000
24      0.00000
25      0.00000
26          NaN
27      0.00000
28      0.00000
29      0.00000
30      0.00000
31      0.00000
32      0.00000
33      0.00000
34      0.00000
35      0.00000
36      0.00000
37      0.00000
38      0.00000
39      0.01000
40    167.94000
41      2.66000
42    112.37000
43    216.89000
44    539.82000
45    158.76000
46    194.32000
47      0.24000
48    121.51000
49      8.24000
50     39.00000
51      9.23000
52      6.34000
53     37.55000
54    143.71000
55      0.00000
Name: Median, dtype: float64